In [1]:
import os
import pickle
import string
import argparse

SAVEDICTIONARYTOPICKLEFILE = 1
USEPICKLEFILE = 1

REBUILDDICTIONARYFROMPICKLEFILE = 0

path = 'F:\LaTeX\LatexWork\Bible'
os.chdir(path)
print (os.getcwd())

import plotly

F:\LaTeX\LatexWork\Bible


In [2]:
# %load BibleWordList2.py
'''
Update 21 Deceber 2018, 0004
'''


import os
import pickle
import collections
import operator
import re

DEBUG = 0

SAVEDICTIONARYTOPICKLEFILE = 1
USEPICKLEFILE = 1
REBUILDDICTIONARYFROMPICKLEFILE = 0

punctuations = '''!()_;:,<>?@#$^.'''

exceptions = [
    'A', 'Accept', 'After', 'Aha', 'All','Alas','Also','Arise',
    'Amen', 'An', 'And', 'As', 'At', 'Awake','According',
    'Be', 'Because', 'Before', 'Behold', 'Blessed','Build','Better',
    'Blessings', 'Blind', 'Both', 'Bring', 'But','Beautiful',
    'By', 'Command', 'Concerning', 'Consider', 'Cut','Come',"Children's",
    'Deal', 'Deceit', 'Deep', 'Deliver', 'Depart','Did',
    'Doubtless', 'Each', 'Either', 'Even', 'Every','Excellent','Fear',
    'For', 'From', 'Get', 'Give', 'Go', 'Grace',
    'Great', 'Hatred', 'Having', 'He', 'Hear',
    'Hearing', 'Heaviness', 'Help', 'His', 'Hold',
    'Horror', 'How', 'If', 'In', 'Incline', 'Judge',
    'Leave', 'Let', 'Lo', 'Look', 'Lying', 'Lodge','Make','Must','Mark',
    'Many', 'Mine', 'My', 'Neither', 'Nevertheless','Nay',
    'Not', 'Now', 'Number', 'O', 'Of', 'On', 'One','Out',
    'Only', 'Open', 'Or', 'Order', 'Our', 'Pardon',
    'Plead', 'Princes', 'Quicken', 'Red', 'Remember',
    'Remove', 'Riches', 'Righteous', 'Rise', 'Rivers','Return',
    'Sacrifice', 'Salvation', 'Say', 'Seven', 'Shall','Spake',
    'So', 'Speak', 'Sprinkle', 'Stablish', 'Stand','Strong',
    'Surely', 'Take', 'Teach', 'That', 'The', 'Their',
    'Then', 'There', 'Therefore', 'These', 'They',
    'This', 'Those', 'Thou', 'Though', 'Through',
    'Thus', 'Thy', 'To', 'Treasures', 'Trouble','Told',
    'Turn', 'Two', 'Unless', 'Uphold', 'We',
    'What', 'Whatsoever', 'When', 'Where', 'Wherefore',
    'Wherewithal', 'Which', 'Who', 'Whom', 'Whoso',
    'Whosoever', 'Why', 'Wise', 'With', 'Withhold','Walk','Wisdom',
    'Would', 'Ye', 'Yea', 'Yet', 'Your']

properNameList = [
    'Aaron', 'Abelmizraim', 'Abiasaph', 'Abib', 'Abihu', 'Abraham', 'Abram',
    'Aceldama', 'Achaia', 'Achaicus', 'Achor', 'Achzib', 'Adam', 'Adramyttium',
    'Adria', 'Adullam', 'Agrippa', 'Ahab', 'Ahaz', 'Ahikam', 'Ahisamach',
    'Aholiab', 'Alexandria', 'Alexandrians', 'Alphaeus', 'Amalek', 'Amittai',
    'Ammi ', 'Amminadab', 'Amon', 'Amorite', 'Amorites', 'Amram', 'Anathoth',
    'Andrew', 'Angel', 'Antioch', 'Aphrah', 'Apollos', 'Apphia', 'Aquila',
    'Arabia ', 'Arabian', 'Archippus', 'Ard', 'Areli', 'Aretas', 'Aristarchus',
    'Arnon', 'Arodi', 'Aroer ', 'Asenath', 'Asenath', 'Ashbel', 'Asher',
    'Ashkelon', 'Asia', 'Assir', 'Assyria ', 'Assyrian', 'Atad', 'Augustus',
    'Azariah', 'Azur', 'BRANCH', 'Baal ', 'Baali ', 'Baalim', 'Baalzephon',
    'Bablyon', 'Balaam', 'Balak', 'Barbarian', 'Barjona', 'Barnabas',
    'Barsabas', 'Bartholomew', 'Baruch', 'Becher', 'Beeri', 'Beerlahairoi',
    'Beersheba', 'Belah', 'Belial', 'Belteshazzar', 'Benjamin', 'Beor',
    'Bered', 'Beriah', 'Bernice', 'Bethaven', 'Bethdiblathaim ', 'Bethel',
    'Bethezel', 'Bethgamul', 'Bethlehem', 'Bethmeon', 'Bethshemesh',
    'Bezaleel', 'Bilhah', 'Bozrah', 'Caesar ', 'Caesarea', 'Cainan', 'Cana',
    'Canaan', 'Canaanite', 'Canaanites', 'Canaanitish', 'Capernaum', 'Caphtor',
    'Carchemish', 'Carmi', 'Castor', 'Cephas', 'Chaldeans', 'Chaldeans',
    'Chemosh', 'Chittim', 'Chloe', 'Christ', 'Cilicia', 'Clauda', 'Cnidus',
    'Crete', 'Cretians', 'Crispus', 'Cyprus', 'Cyrenians', 'Cyrus',
    'Damascenes', 'Damascus', 'Dan', 'Daniel', 'Darius', 'David', 'Demas',
    'Diblaim', 'Dibon', 'Didymus', 'Dinah', 'Edom', 'Egypt', 'Egyptian',
    'Egyptians', 'Ehi', 'Elasah', 'Eleazar', 'Elias', 'Eliezer', 'Elim',
    'Elisheba', 'Elkanah', 'Elon', 'Elzaphan', 'Enoch', 'Enos', 'Epaphras',
    'Ephesus', 'Ephraim', 'Ephrath', 'Ephron', 'Er', 'Eri', 'Esaias', 'Esau',
    'Etham', 'Euphrates', 'Euroclydon', 'Ezbon', 'Father', 'Felix', 'Festus',
    'Fortunatus', 'Gad', 'Gadarenes', 'Gaius', 'Galatia', 'Galilee', 'Gath',
    'Gaza', 'Gedaliah', 'Gemariah', 'Gentiles', 'Gera', 'Gershom', 'Gershon',
    'Gibeah', 'Gibeon', 'Gilead', 'Gilgal', 'God', 'Gomer', 'Goshen',
    'Grecians', 'Greek', 'Guni', 'Hagar', 'Haggi', 'Ham', 'Hananiah', 'Hanoch',
    'Heber', 'Hebrew', 'Hebrews', "Hebrews'", 'Hebron', 'Heldai', 'Helem',
    'Hen', 'Heshbon', 'Hezekiah', 'Hezron', 'High', 'Hilkiah', 'Hinnom',
    'Hittite', 'Hittites', 'Hivite', 'Hivites', 'Horeb', 'Horonaim', 'Hosea',
    'Hoshaiah', 'Huppim', 'Hur', 'Hushim', 'I', 'Isaac', 'Ishi', 'Ishmael',
    'Ishmeelites', 'Ishuah', 'Israel', 'Issachar', 'Issachar', 'Isui', 'Italy',
    'Ithamar', 'Ithiel', 'Izhar', 'JEHOVAH', 'Jachin', 'Jacob', 'Jahleel',
    'Jahzeel', 'Jairus', 'Jakeh', 'James', 'Jamin', 'Japheth', 'Jareb:',
    'Jared', 'Jebusite', 'Jebusites', 'Jeconiah', 'Jedaiah', 'Jehoiada',
    'Jehoiakim', 'Jehoshapha', 'Jehovahnissi', 'Jehu', 'Jemuel', 'Jeremiah',
    'Jerusalem', 'Jesus', 'Jethro', 'Jews', 'Jezer', 'Jezreel', 'Jimnah',
    'Joash', 'Job', 'Jochebed', 'Joel', 'Johanan', 'John', 'Jonah', 'Jonas',
    'Joppa', 'Jordan', 'Josedech', 'Joseph', 'Josiah', 'Jotham', 'Judaea',
    'Judah', 'Judas', 'Julius', 'Just ', 'Justus ', 'Kadesh', 'Kareah',
    'Kedar', 'Kerioth', 'King', 'Kiriathaim ', 'Kohath', 'Korah', 'Korhites',
    'LORD', 'Laban', 'Laban', 'Lachis', 'Lamech', 'Laodicea', 'Lasea', 'Leah',
    'Lebanon', 'Legion', 'Levi', 'Libertines', 'Libni', 'Libyans', 'Loammi',
    'Lord', 'Lord Jesus Christ', 'Loruhamah', 'Lucas', 'Luhith', 'Luz', 'Lycia',
    'Lydians', 'Maaseiah', 'Macedonia', 'Macedonian', 'Machir', 'Machpelah',
    'Mahalaleel', 'Mahali', 'Malachi', 'Malchiel', 'Mamre', 'Manasseh',
    'Marcus', 'Mareshah', 'Maroth', 'Massah', 'Matthew', 'Matthias',
    'Melchizedek', 'Melita', 'Melzar', 'Merari', 'Meribah', 'Messias',
    'Methuselah', 'Micah', 'Midian', 'Migdol', 'Miriam', 'Misgab', 'Mishael',
    'Mizpah', 'Moab', 'Morasthite', 'Moreshethgath', 'Moses', 'Muppim', 'Mushi',
    'Myra', 'Naaman', 'Naashon', 'Nadab', 'Naphtali', 'Nathanael', 'Nazareth',
    'Nebo', 'Nebuchadnezzar', 'Nebuchadrezzar', 'Nebuzaradan', 'Nehelamite',
    'Nepheg', 'Neriah', 'Nicanor', 'Nicolas', 'Nicopolis', 'Nineveh', 'No ',
    'Noah', 'Noph', 'Ohad', 'Olivet', 'Omri', 'Onan', 'Onesimus', 'Padan',
    'Padanaram', 'Palestine', 'Pallu', 'Pamphylia', 'Parmenas', 'Pashur',
    'Pathros', 'Paul', 'Pentecost', 'Pergamos', 'Perizzite', 'Perizzites',
    'Peter', 'Pethuel', 'Phallu', 'Pharaoh', "Pharaoh's", 'Pharaohhophra ',
    'Pharaohnecho', 'Pharez', 'Pharisee ', 'Pharisees', 'Phenice',
    'Philadelphia', 'Philemon', 'Philip', 'Philippi', 'Philistines',
    'Phinehas', 'Phuvah', 'Pihahiroth', 'Pilate', 'Pithom', 'Pollux',
    'Potiphar', 'Potipherah', 'Prince', 'Priscilla', 'Prochorus', 'Puah',
    'Publius', 'Puteoli', 'Putiel', 'Raamses', 'Rachel', 'Ramah', 'Rameses',
    'Rephidim', 'Reuben', 'Reuel', 'Rhegium', 'Rock', 'Rome', 'Rosh', 'Ruhamah',
    'Sabeans', 'Sadducees', 'Salem', 'Salmone', 'Samaria', 'Samaritans',
    'Samuel', 'Saphir', 'Sara', 'Sarah', 'Sarai', 'Sardis', 'Satan', 'Saul',
    'Saviour', 'Scythian', 'Selah', 'Sepharad', 'Serah', 'Sered', 'Seth',
    'Shaphan', 'Shaul', 'Shelah', 'Shem', 'Shemaiah', 'Shillem', 'Shiloh',
    'Shimi', 'Shimron', 'Shiphrah', 'Shittim', 'Shuni', 'Sidon', 'Sihon',
    'Sihor', 'Simeon', 'Simon ', 'Simon Zelotes', 'Sinai', 'Sirion', 'Smyrna',
    'Solomon', 'Sosthenes', 'Stephanas', 'Stephen', 'Succoth', 'Sychar',
    'Syracuse', 'Tabor', 'Tahpanhes', 'Tarshish', 'Teman', 'Theophilus',
    'Thessalonica', 'Thomas', 'Thummim', 'Thyatira', 'Tiberias', 'Timon',
    'Timotheus', 'Timothy', 'Titus', 'Tobijah', 'Tola', 'Tophet', 'Tychicus',
    'Tyre', 'Tyrus', 'Ucal', 'Uri', 'Urim', 'Uzziah', 'Uzziel', 'Zaanan',
    'Zaphnathpaaneah', 'Zarephath', 'Zebedee', 'Zebulun', 'Zedekiah',
    'Zephaniah', 'Zerah', 'Zichri', 'Zidon', 'Zilpah', 'Zion', 'Ziphion',
    'Zipporah', 'Zithri', 'Zohar']

##########
def removePunc(input):
    no_punc = ''
    for char in input:
        if char not in punctuations:
            no_punc = no_punc + char
    return no_punc


##########
def generatedLatexFormattedText(dict):
    BUILDAWIP = 1
    BUILDNWIV = 1
    BUILDPNIP = 1
    BUILDNWIC = 1
    BUILDNLIW = 1
    BUILDBPIP = 1
    MINISTRY = 1

    topString = "\\footnote{\\textcolor[cmyk]{0.99998,1,0,0}{\\hyperlink{TOC}{Return to end " \
                "of Table of Contents.}}}"
    textColorString = "\\textcolor[cmyk]{0.99998,1,0,0}{"
    endString = "}"

    '''
    Print plain text with Latex color header:
        \textcolor[rgb]{0.00,0.00,1.00} == BLUE
        \textcolor[cmyk]{0.99998,1,0,0} == A DEEPER BLUE
    '''
    print('\n\n%%%%%%%%%%%%%%%%%%%%%%%%%%%')
    print('Chapter with Latex format\n%%%%%%%%%%%%%%%%%%%%%%%%%%%\n\n')

    linesWith13Words = []
    linesWith18Words = []
    for line in dict:
        booknm = line[0]
        chapnm = line[1]
        chapno = line[2]
        versno = line[3]
        versn2 = line[4]
        newline = line[5]
        if versn2 == '1':
            print(topString + textColorString + newline + endString)
        else:
            print('[' + versn2 + '] ' + textColorString + newline + endString)
        words = newline.split()
        wordLength = len(words)
        if wordLength == 13:
            linesWith13Words.append(line)
        if wordLength == 18:
            linesWith18Words.append(line)
            
        
    '''
    NOW print the verses with LaTeX index items:
        AWIP - All Words in Passage
        NWIV - NUmber of Words in Verse
        PNIP - Proper noun in passage
        ... more later
    '''
    print('\n\n%%%%%%%%%%%%%%%%%%%%%%%%%%%')
    print('Chapter with indexed items\n%%%%%%%%%%%%%%%%%%%%%%%%%%%\n\n')
    for line in dict:
        booknm = line[0]
        chapnm = line[1]
        chapno = line[2]
        versno = line[3]
        versn2 = line[4]
        verseRef = "!" + booknm + "!" + chapnm + " " + chapno + ":" + versno 
        line = line[5]
        if versn2 == '1':
            verseString = topString + textColorString + line + endString
        else:
            verseString = '[' + versn2 + '] ' + textColorString + line + endString
        #print(booknm,chapnm,chapno,versno,versn2) 
        '''
        NWIV
        '''
        words = line.split()
        numWords = len(words)
        if DEBUG == 1:
            print(numWords,'Words in verse',versn2)
        extraString="\\index[NWIV]{" + str(numWords) + verseRef + "}"
        '''
        AWIP - here, Passage = verse
        '''
        verseDict = {}
        wordString = ''
        for word in words:
            word = removePunc(word)
            if word not in verseDict:
                verseDict[word] = 1
                wordString = wordString + "\\index[AWIP]{" + word + verseRef + "}"
            else:
                verseDict[word] = verseDict[word] + 1
                wordString = wordString + "\\index[AWIP]{" + word + verseRef 
                wordString = wordString + " (" + str(verseDict[word]) + ")}"
        '''
        PNIP - Proper names in passage
        '''
        propernameStrng = ''
        for item in verseDict:
            if item in properNameList:
                if item not in exceptions:
                    propernameStrng = propernameStrng + "\\index[PNIP]{" + item + verseRef + "}"

        builtVerseString = verseString + extraString + wordString  + propernameStrng
        print(builtVerseString)
    return linesWith18Words, linesWith13Words

##########
def showWordsByFrequency(dictionary,arg):
    r = re.compile("([a-zA-Z]+)([0-9]+)")
    m = r.match(arg)

    print('\\subsection{Words by Frequency}')
    print ("\\scriptsize")
    print ("\\begin{center}")
    print ("\\begin{longtable}{l|r}")
    print ("\caption[Word Frequencies in "+m.group(1)+" "+str(int(m.group(2)))+"]{Word Frequencies in "+m.group(1)+" "+str(int(m.group(2)))+"} \label{table:WordsIn-"+m.group(1)+"-"+str(int(m.group(2)))+"} \\\ ")    
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Word}} & \multicolumn{1}{c|}{\\textbf{Frequency}} \\\ \\hline ")
    print ("\\endfirsthead")
    print (" ")
    print ("\\multicolumn{2}{c}")
    print ("{{\\bfseries \\tablename\ \\thetable{} -- continued from previous page}} \\\ ")
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Word}} & \multicolumn{1}{c|}{\\textbf{Frequency}} \\\ \\hline ")
    print ("\\endhead")
    print (" ")
    print ("\hline \multicolumn{2}{|r|}{{Continued if needed}} \\\ \\hline")
    print ("\\endfoot")
    print (" ")
    print ("\hline \hline")
    print ("\\endlastfoot")
    sortedDictionary = sorted(dictionary.items(), key=lambda x: x[1], reverse=True)
    for item in sortedDictionary:
        print (item[0], "&", item[1], "\\\ \hline")
    print ("\\end{longtable}")
    print ("\\end{center}")
    print ("\n\n")
    print ("\\normalsize")
    print ("\n\n")    

##########
def showWordsAlphabetically(dictionary,arg):
    r = re.compile("([a-zA-Z]+)([0-9]+)")
    m = r.match(arg)

    print('\\subsection{Words Alphabetically}')
    print ("\\scriptsize")
    print ("\\begin{center}")
    print ("\\begin{longtable}{l|r}")
    print ("\caption[Word Alphabetically in "+m.group(1)+" "+str(int(m.group(2)))+"]{Word Alphabetically in "+m.group(1)+" "+str(int(m.group(2)))+"} \label{table:WordsIn-"+m.group(1)+"-"+str(int(m.group(2)))+"} \\\ ")    
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Word}} & \multicolumn{1}{c|}{\\textbf{Frequency}} \\\ \\hline ")
    print ("\\endfirsthead")
    print (" ")
    print ("\\multicolumn{2}{c}")
    print ("{{\\bfseries \\tablename\ \\thetable{} -- continued from previous page}} \\\ ")
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Word}} & \multicolumn{1}{c|}{\\textbf{Frequency}} \\\ \\hline ")
    print ("\\endhead")
    print (" ")
    print ("\hline \multicolumn{2}{|r|}{{Continued if needed}} \\\ \\hline")
    print ("\\endfoot")
    print (" ")
    print ("\hline \hline")
    print ("\\endlastfoot")
    sortedDictionary = sorted(dictionary.items(), key=lambda x: x[0], reverse=False)
    for item in sortedDictionary:
        print (item[0], "&", item[1], "\\\ \hline")
    print ("\\end{longtable}")
    print ("\\end{center}")
    print ("\n\n")
    print ("\\normalsize")
    print ("\n\n")    

##########
def generateWordStatisticsForChapter(label,verses):
    '''
    Need to simplify this code! ONLY count number of words, number of unique 
    words, number of itallic words, number of unique itallic words
    
    verseDictionary, rest for each verse
    chapterDictionary
    '''
    global properNameList
    wordCount = 0
    emphasizedwordCount = 0
    wordDict = {}
    emphasizedwordDict = {}
    verseString = ''
    A = re.findall(r'\d+', label)
    A = " ".join(map(str, A))
    digit = int(A)

    word1 = " ".join(re.findall("[a-zA-Z]+", label))
    
    print('\n\n%%%%%%%%%%%%%%%%%%%%%%%%%%%')
    print('%%%%% Word Statistics\n%%%%%%%%%%%%%%%%%%%%%%%%%%\n\n')
    
    print('\\normalsize')

    verseStats = ["\\subsection{Chapter Word Statistics}",
                  "\n",
                  "%%%%%%%%%%",
                  "%%%%%%%%%%",
                  " ",
                  "\\begin{center}",
                  "\\begin{longtable}{l|c|c|c|c}",
                  "\\caption[Stats for "+word1+" "+str(digit) +"]{Stats for "+word1+" "+str(digit) +"} \\label{table:Stats for "+word1+" "+str(digit) +"} \\\ ",
                  "\\hline \multicolumn{1}{|c|}{\\textbf{Verse(s)}} & \multicolumn{1}{|c|}{\\textbf{Count}} & \multicolumn{1}{|c|}{\\textbf{Unique}} & \multicolumn{1}{|c|}{\\textbf{Italics}} & \multicolumn{1}{|c|}{\\textbf{Uniq Italic}}  \\\ \\hline ",
                  "\\endfirsthead",
                  " ",
                  "\multicolumn{5}{c}",
                  "{{\\bfseries \\tablename\ \\thetable{} -- continued from previous page}} \\\  ",
                  "\\hline \multicolumn{1}{|c|}{\\textbf{Verse(s)}} & \multicolumn{1}{|c|}{\\textbf{Count}} & \multicolumn{1}{|c|}{\\textbf{Unique}} & \multicolumn{1}{|c|}{\\textbf{Italics}} & \multicolumn{1}{|c|}{\\textbf{Uniq Italic}}  \\\ \\hline ",
                  "\\endhead",
                  " ",
                  "\\hline \\multicolumn{5}{|r|}{{Continued if needed}} \\\ \\hline",
                  "\\endfoot ",]

    '''
    This section prints JUST the Bible text in a LaTeX format with a textcolor encoding added
    Original Code
    '''
    chapterDict = {}
    chapwordWords = 0
    chapterUniqueWords = 0
    numberChapterUniqueItallics = 0
    numberChapterItallics = 0
    verseNo = 0
    for line in verses:
        verseNo += 1
        verseDict = {}
#        print(line)
        words = removePunc(line[5])
        wordcount = 0
        uniqueWords = 0        
        '''
        Process for verse
        '''
        for word in words.split():
            wordcount += 1
            if word not in verseDict:
                verseDict[word] = 1
                uniqueWords += 1
            else:
                verseDict[word] = verseDict[word] + 1
        numberUniqueItallics = 0
        numberItallics = 0
        for item in verseDict:
            if "\emph{" in item:
                numberUniqueItallics += 1
                numberItallics = numberItallics + verseDict[item]                
        '''
        Process for chapter
        '''                
        for word in words.split():
            chapwordWords += 1
            if word not in chapterDict:
                chapterDict[word] = 1
                chapterUniqueWords += 1
            else:
                chapterDict[word] = chapterDict[word] + 1
        sequence = str(verseNo),str(wordcount),str(uniqueWords),str(numberItallics),str(numberUniqueItallics)+'\\\ \hline'
        S = ' & '.join(sequence)
        verseStats.append(S)
    for item in chapterDict:
        if "\emph{" in item:
            numberChapterUniqueItallics +=1
            numberChapterItallics = numberChapterItallics + chapterDict[item]

    '''
    Finish Up!
    '''
#    print ("\n",wordCount,"words in passage, of which",len(wordDict),"are unique")
#    print (emphasizedwordCount,'words in ital, of which',len(emphasizedwordDict),'are unique')
    print ("\n\n")
    verseStats.append("\hline \hline")
    sequence = 'Total',str(chapwordWords),str(chapterUniqueWords),str(numberChapterItallics),str(numberChapterUniqueItallics)
    S = ' & '.join(sequence)
    verseStats.append(S)
    for item in verseStats:
        print (item)
    print ("\n\n")
    print ("\\end{longtable}")
    print ("\\end{center}\n")
#    print (chapwordWords,"words in passage, of which",chapterUniqueWords,"are unique")
#    print (numberChapterItallics,'words in ital, of which',numberChapterUniqueItallics,'are unique')
#    print ("\n\n")
#
    print ("%%%%%%%%%%\n%%%%%%%%%%")
    print (" ")
    
    return chapterDict

##########
def getWordLengthsInChapter(dictionary,arg):
    r = re.compile("([a-zA-Z]+)([0-9]+)")
    m = r.match(arg)

    wordLengthDictionary = {}
    for item in dictionary:
        if "\emph{" in item:
            tempitem = item.replace("\emph{",'')
            tempitem = tempitem.replace('}','')
            if len(tempitem) in wordLengthDictionary:
                wordLengthDictionary[len(tempitem)] = wordLengthDictionary[len(tempitem)] + ', ' + item
            else:
                    wordLengthDictionary[len(tempitem)] = item
        else:
            if len(item) in wordLengthDictionary:
                wordLengthDictionary[len(item)] = wordLengthDictionary[len(item)] + ', ' + item
            else:
                wordLengthDictionary[len(item)] = item
    wordLengthDictionary = sorted(wordLengthDictionary.items(), key=lambda x:x[0], reverse=False)

    print('\\subsection{Word Lengths in Chapter}')
    print ("\\normalsize")
#    print ("\\begin{center}")
    print ("\\begin{longtable}{l|p{3.75in}}")
    print ("\caption[Words by Length in "+m.group(1)+" "+str(int(m.group(2)))+"]{Words by Length in "+m.group(1)+" "+str(int(m.group(2)))+"} \label{table:WordsIn-"+m.group(1)+"-"+str(int(m.group(2)))+"} \\\ ")    
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Length}} & \multicolumn{1}{c|}{\\textbf{Words}} \\\ \\hline ")
    print ("\\endfirsthead")
    print (" ")
    print ("\\multicolumn{2}{c}")
    print ("{{\\bfseries \\tablename\ \\thetable{} -- continued from previous page}} \\\ ")
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Length}} & \multicolumn{1}{c|}{\\textbf{Words}} \\\ \\hline ")
    print ("\\endhead")
    print (" ")
    print ("\hline \multicolumn{2}{|r|}{{Continued if needed}} \\\ \\hline")
    print ("\\endfoot")
    print (" ")
    print ("\hline \hline")
    print ("\\endlastfoot")
    for item in wordLengthDictionary:
        print (item[0], "&", item[1], "\\\ \hline")
    print ("\\end{longtable}")
#    print ("\\end{center}")
    print ("\n\n")
    print ("\n\n") 
    print ("%%%%%%%%%%\n%%%%%%%%%%")
    print (" ")

##########
def makeVersesWith13WordsTable(list,arg):
    print ("\n\n") 
    print ("%%%%%%%%%%\n%%%%%%%%%%")

    r = re.compile("([a-zA-Z]+)([0-9]+)")
    m = r.match(arg)


    print('\\subsection{Verses with 13 Words in Chapter}')
    print ("\\normalsize")
#    print ("\\begin{center}")
    print ("\\begin{longtable}{l|p{3.75in}}")
    print ("\caption[Verses with 13 Words  in "+m.group(1)+" "+str(int(m.group(2)))+"]{Verses with 13 Words  in "+m.group(1)+" "+str(int(m.group(2)))+"} \label{table:Verses with 13 Words in-"+m.group(1)+"-"+str(int(m.group(2)))+"} \\\ ")    
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Reference}} & \multicolumn{1}{c|}{\\textbf{Verse}} \\\ \\hline ")
    print ("\\endfirsthead")
    print (" ")
    print ("\\multicolumn{2}{c}")
    print ("{{\\bfseries \\tablename\ \\thetable{} -- continued from previous page}} \\\ ")
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Reference}} & \multicolumn{1}{c|}{\\textbf{Verse}} \\\ \\hline ")
    print ("\\endhead")
    print (" ")
    print ("\hline \multicolumn{2}{|r|}{{Continued if needed}} \\\ \\hline")
    print ("\\endfoot")
    print (" ")
    print ("\hline \hline")
    print ("\\endlastfoot")
    for item in list:
        print (item[0]+' '+item[2]+":"+item[4], "&", item[5], "\\\ \hline")
    print ("\\end{longtable}")
#    print ("\\end{center}")
    print ("\n\n")
    print ("\n\n") 
    print ("%%%%%%%%%%\n%%%%%%%%%%")
    print (" ")
        
##########
          
##########
def makeVersesWith18WordsTable(list,arg):
    print ("\n\n") 
    print ("%%%%%%%%%%\n%%%%%%%%%%")

    r = re.compile("([a-zA-Z]+)([0-9]+)")
    m = r.match(arg)


    print('\\subsection{Verses with 18 Words in Chapter}')
    print ("\\normalsize")
#    print ("\\begin{center}")
    print ("\\begin{longtable}{l|p{3.75in}}")
    print ("\caption[Verses with 18 Words  in "+m.group(1)+" "+str(int(m.group(2)))+"]{Verses with 18 Words  in "+m.group(1)+" "+str(int(m.group(2)))+"} \label{table:Verses with 18 Words in-"+m.group(1)+"-"+str(int(m.group(2)))+"} \\\ ")    
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Reference}} & \multicolumn{1}{c|}{\\textbf{Verse}} \\\ \\hline ")
    print ("\\endfirsthead")
    print (" ")
    print ("\\multicolumn{2}{c}")
    print ("{{\\bfseries \\tablename\ \\thetable{} -- continued from previous page}} \\\ ")
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Reference}} & \multicolumn{1}{c|}{\\textbf{Verse}} \\\ \\hline ")
    print ("\\endhead")
    print (" ")
    print ("\hline \multicolumn{2}{|r|}{{Continued if needed}} \\\ \\hline")
    print ("\\endfoot")
    print (" ")
    print ("\hline \hline")
    print ("\\endlastfoot")
    for item in list:
        print (item[0]+' '+item[2]+":"+item[4], "&", item[5], "\\\ \hline")
    print ("\\end{longtable}")
#    print ("\\end{center}")
    print ("\n\n")
    print ("\n\n") 
    print ("%%%%%%%%%%\n%%%%%%%%%%")
    print (" ")
        
##########

def countLettersInVerses(arg1,arg2):
    for item in arg2:
        letterDict = {}
        for item2 in item[5]:
            if "\emph" in item2:
                item2 = item2.replace('\emph','')
            if item2 != ' ':
                if item2 in letterDict:
                    letterDict[item2] = letterDict[item2] + 1
                else:
                    letterDict[item2] =  1
            print ("for verse",item[2],":",letterDict)

def ProcessVerses(arg1,arg2):
    ''' 
    arg1 is dictionary name and arg2 is the dictionary
    '''
#    saveToPickleFile(arg1,arg2)  
    linesWith18words, linesWith13words = generatedLatexFormattedText(arg2)         # format chapter in Latex
    #linesWith13words is for use later ... 
    chapterDict = generateWordStatisticsForChapter(arg1,arg2)   # do chapter word statistics
    showWordsByFrequency(chapterDict,arg1)                      # show words by frequency
    showWordsAlphabetically(chapterDict,arg1)                   # show words alphabetically
    getWordLengthsInChapter(chapterDict,arg1)                   # show word lengths in chapter
    
    if len(linesWith13words) > 0:
        #print(linesWith13words)
        makeVersesWith13WordsTable(linesWith13words,arg1)
        
    if len(linesWith18words) > 0:
        #print(linesWith13words)
        makeVersesWith18WordsTable(linesWith18words,arg1)
        
    ''' In Development
    countLettersInVerses(arg1,arg2)
    '''



In [3]:
path = 'F:\python'
os.chdir(path)
print (os.getcwd())

F:\python


In [4]:
# %load NEW21OT-MakeConcordance-Ecclesiastes.py
"""
Created on Sun Jan 01 07:52:18 2017

@author: keith
"""
import os
import pickle
import string

from BibleWordList2 import *

BUILDAWIP = 1
BUILDNWIV = 1
BUILDPNIP = 1
BUILDNWIC = 1
BUILDNLIW = 1
BUILDBPIP = 1

SAVEDICTIONARYTOPICKLEFILE = 1
USEPICKLEFILE = 1
REBUILDDICTIONARYFROMPICKLEFILE = 0

from BibleWordList2 import *

Ecclesiastes01 = [
    ['Ecclesiastes','Ecc','01','001','1',"The words of the Preacher, the son of David, king in Jerusalem."],
    ['Ecclesiastes','Ecc','01','002','2',"Vanity of vanities, saith the Preacher, vanity of vanities; all \emph{is} vanity."],
    ['Ecclesiastes','Ecc','01','003','3',"What profit hath a man of all his labour which he taketh under the sun?"],
    ['Ecclesiastes','Ecc','01','004','4',"\emph{One} generation passeth away, and \emph{another} generation cometh: but the earth abideth for ever."],
    ['Ecclesiastes','Ecc','01','005','5',"The sun also ariseth, and the sun goeth down, and hasteth to his place where he arose."],
    ['Ecclesiastes','Ecc','01','006','6',"The wind goeth toward the south, and turneth about unto the north; it whirleth about continually, and the wind returneth again according to his circuits."],
    ['Ecclesiastes','Ecc','01','007','7',"All the rivers run into the sea; yet the sea \emph{is} not full; unto the place from whence the rivers come, thither they return again."],
    ['Ecclesiastes','Ecc','01','008','8',"All things \emph{are} full of labour; man cannot utter \emph{it}: the eye is not satisfied with seeing, nor the ear filled with hearing."],
    ['Ecclesiastes','Ecc','01','009','9',"The thing that hath been, it \emph{is} \emph{that} which shall be; and that which is done \emph{is} that which shall be done: and \emph{there} \emph{is} no new \emph{thing} under the sun."],
    ['Ecclesiastes','Ecc','01','010','10',"Is there \emph{any} thing whereof it may be said, See, this \emph{is} new? it hath been already of old time, which was before us."],
    ['Ecclesiastes','Ecc','01','011','11',"\emph{There} \emph{is} no remembrance of former \emph{things}; neither shall there be \emph{any} remembrance of \emph{things} that are to come with \emph{those} that shall come after."],
    ['Ecclesiastes','Ecc','01','012','12',"I the Preacher was king over Israel in Jerusalem."],
    ['Ecclesiastes','Ecc','01','013','13',"And I gave my heart to seek and search out by wisdom concerning all \emph{things} that are done under heaven: this sore travail hath God given to the sons of man to be exercised therewith."],
    ['Ecclesiastes','Ecc','01','014','14',"I have seen all the works that are done under the sun; and, behold, all \emph{is} vanity and vexation of spirit."],
    ['Ecclesiastes','Ecc','01','015','15',"\emph{That} \emph{which} \emph{is} crooked cannot be made straight: and that which is wanting cannot be numbered."],
    ['Ecclesiastes','Ecc','01','016','16',"I communed with mine own heart, saying, Lo, I am come to great estate, and have gotten more wisdom than all \emph{they} that have been before me in Jerusalem: yea, my heart had great experience of wisdom and knowledge."],
    ['Ecclesiastes','Ecc','01','017','17',"And I gave my heart to know wisdom, and to know madness and folly: I perceived that this also is vexation of spirit."],
    ['Ecclesiastes','Ecc','01','018','18',"For in much wisdom \emph{is} much grief: and he that increaseth knowledge increaseth sorrow."]
]


Ecclesiastes02 = [
['Ecclesiastes','Ecc','02','001','1',"I said in mine heart, Go to now, I will prove thee with mirth, therefore enjoy pleasure: and, behold, this also \emph{is} vanity."],
['Ecclesiastes','Ecc','02','002','2',"I said of laughter, \emph{It} \emph{is} mad: and of mirth, What doeth it?"],
['Ecclesiastes','Ecc','02','003','3',"I sought in mine heart to give myself unto wine, yet acquainting mine heart with wisdom; and to lay hold on folly, till I might see what \emph{was} that good for the sons of men, which they should do under the heaven all the days of their life."],
['Ecclesiastes','Ecc','02','004','4',"I made me great works; I builded me houses; I planted me vineyards:"],
['Ecclesiastes','Ecc','02','005','5',"I made me gardens and orchards, and I planted trees in them of all \emph{kind} \emph{of} fruits:"],
['Ecclesiastes','Ecc','02','006','6',"I made me pools of water, to water therewith the wood that bringeth forth trees:"],
['Ecclesiastes','Ecc','02','007','7',"I got \emph{me} servants and maidens, and had servants born in my house; also I had great possessions of great and small cattle above all that were in Jerusalem before me:"],
['Ecclesiastes','Ecc','02','008','8',"I gathered me also silver and gold, and the peculiar treasure of kings and of the provinces: I gat me men singers and women singers, and the delights of the sons of men, \emph{as} musical instruments, and that of all sorts."],
['Ecclesiastes','Ecc','02','009','9',"So I was great, and increased more than all that were before me in Jerusalem: also my wisdom remained with me."],
['Ecclesiastes','Ecc','02','010','10',"And whatsoever mine eyes desired I kept not from them, I withheld not my heart from any joy; for my heart rejoiced in all my labour: and this was my portion of all my labour."],
['Ecclesiastes','Ecc','02','011','11',"Then I looked on all the works that my hands had wrought, and on the labour that I had laboured to do: and, behold, all \emph{was} vanity and vexation of spirit, and \emph{there} \emph{was} no profit under the sun."],
['Ecclesiastes','Ecc','02','012','12',"And I turned myself to behold wisdom, and madness, and folly: for what \emph{can} the man \emph{do} that cometh after the king? \emph{even} that which hath been already done."],
['Ecclesiastes','Ecc','02','013','13',"Then I saw that wisdom excelleth folly, as far as light excelleth darkness."],
['Ecclesiastes','Ecc','02','014','14',"The wise man's eyes \emph{are} in his head; but the fool walketh in darkness: and I myself perceived also that one event happeneth to them all."],
['Ecclesiastes','Ecc','02','015','15',"Then said I in my heart, As it happeneth to the fool, so it happeneth even to me; and why was I then more wise? Then I said in my heart, that this also \emph{is} vanity."],
['Ecclesiastes','Ecc','02','016','16',"For \emph{there} \emph{is} no remembrance of the wise more than of the fool for ever; seeing that which now \emph{is} in the days to come shall all be forgotten. And how dieth the wise \emph{man}? as the fool."],
['Ecclesiastes','Ecc','02','017','17',"Therefore I hated life; because the work that is wrought under the sun \emph{is} grievous unto me: for all \emph{is} vanity and vexation of spirit."],
['Ecclesiastes','Ecc','02','018','18',"Yea, I hated all my labour which I had taken under the sun: because I should leave it unto the man that shall be after me."],
['Ecclesiastes','Ecc','02','019','19',"And who knoweth whether he shall be a wise \emph{man} or a fool? yet shall he have rule over all my labour wherein I have laboured, and wherein I have shewed myself wise under the sun. This \emph{is} also vanity."],
['Ecclesiastes','Ecc','02','020','20',"Therefore I went about to cause my heart to despair of all the labour which I took under the sun."],
['Ecclesiastes','Ecc','02','021','21',"For there is a man whose labour \emph{is} in wisdom, and in knowledge, and in equity; yet to a man that hath not laboured therein shall he leave it \emph{for} his portion. This also \emph{is} vanity and a great evil."],
['Ecclesiastes','Ecc','02','022','22',"For what hath man of all his labour, and of the vexation of his heart, wherein he hath laboured under the sun?"],
['Ecclesiastes','Ecc','02','023','23',"For all his days \emph{are} sorrows, and his travail grief; yea, his heart taketh not rest in the night. This is also vanity."],
['Ecclesiastes','Ecc','02','024','24',"\emph{There} \emph{is} nothing better for a man, \emph{than} that he should eat and drink, and \emph{that} he should make his soul enjoy good in his labour. This also I saw, that it \emph{was} from the hand of God."],
['Ecclesiastes','Ecc','02','025','25',"For who can eat, or who else can hasten \emph{hereunto}, more than I?"],
['Ecclesiastes','Ecc','02','026','26',"For \emph{God} giveth to a man that \emph{is} good in his sight wisdom, and knowledge, and joy: but to the sinner he giveth travail, to gather and to heap up, that he may give to \emph{him} \emph{that} \emph{is} good before God. This also \emph{is} vanity and vexation of spirit."]
]

Ecclesiastes03 = [
['Ecclesiastes','Ecc','03','001','1',"To every \emph{thing} \emph{there} \emph{is} a season, and a time to every purpose under the heaven:"],
['Ecclesiastes','Ecc','03','002','2',"A time to be born, and a time to die; a time to plant, and a time to pluck up \emph{that} \emph{which} \emph{is} planted;"],
['Ecclesiastes','Ecc','03','003','3',"A time to kill, and a time to heal; a time to break down, and a time to build up;"],
['Ecclesiastes','Ecc','03','004','4',"A time to weep, and a time to laugh; a time to mourn, and a time to dance;"],
['Ecclesiastes','Ecc','03','005','5',"A time to cast away stones, and a time to gather stones together; a time to embrace, and a time to refrain from embracing;"],
['Ecclesiastes','Ecc','03','006','6',"A time to get, and a time to lose; a time to keep, and a time to cast away;"],
['Ecclesiastes','Ecc','03','007','7',"A time to rend, and a time to sew; a time to keep silence, and a time to speak;"],
['Ecclesiastes','Ecc','03','008','8',"A time to love, and a time to hate; a time of war, and a time of peace."],
['Ecclesiastes','Ecc','03','009','9',"What profit hath he that worketh in that wherein he laboureth?"],
['Ecclesiastes','Ecc','03','010','10',"I have seen the travail, which God hath given to the sons of men to be exercised in it."],
['Ecclesiastes','Ecc','03','011','11',"He hath made every \emph{thing} beautiful in his time: also he hath set the world in their heart, so that no man can find out the work that God maketh from the beginning to the end."],
['Ecclesiastes','Ecc','03','012','12',"I know that \emph{there} \emph{is} no good in them, but for \emph{a} \emph{man} to rejoice, and to do good in his life."],
['Ecclesiastes','Ecc','03','013','13',"And also that every man should eat and drink, and enjoy the good of all his labour, it \emph{is} the gift of God."],
['Ecclesiastes','Ecc','03','014','14',"I know that, whatsoever God doeth, it shall be for ever: nothing can be put to it, nor any thing taken from it: and God doeth \emph{it}, that \emph{men} should fear before him."],
['Ecclesiastes','Ecc','03','015','15',"That which hath been is now; and that which is to be hath already been; and God requireth that which is past."],
['Ecclesiastes','Ecc','03','016','16',"And moreover I saw under the sun the place of judgment, \emph{that} wickedness \emph{was} there; and the place of righteousness, \emph{that} iniquity \emph{was} there."],
['Ecclesiastes','Ecc','03','017','17',"I said in mine heart, God shall judge the righteous and the wicked: for \emph{there} \emph{is} a time there for every purpose and for every work."],
['Ecclesiastes','Ecc','03','018','18',"I said in mine heart concerning the estate of the sons of men, that God might manifest them, and that they might see that they themselves are beasts."],
['Ecclesiastes','Ecc','03','019','19',"For that which befalleth the sons of men befalleth beasts; even one thing befalleth them: as the one dieth, so dieth the other; yea, they have all one breath; so that a man hath no preeminence above a beast: for all \emph{is} vanity."],
['Ecclesiastes','Ecc','03','020','20',"All go unto one place; all are of the dust, and all turn to dust again."],
['Ecclesiastes','Ecc','03','021','21',"Who knoweth the spirit of man that goeth upward, and the spirit of the beast that goeth downward to the earth?"],
['Ecclesiastes','Ecc','03','022','22',"Wherefore I perceive that \emph{there} \emph{is} nothing better, than that a man should rejoice in his own works; for that \emph{is} his portion: for who shall bring him to see what shall be after him?"]
]

Ecclesiastes04 = [
['Ecclesiastes','Ecc','04','001','1',"So I returned, and considered all the oppressions that are done under the sun: and behold the tears of \emph{such} \emph{as} \emph{were} oppressed, and they had no comforter; and on the side of their oppressors \emph{there} \emph{was} power; but they had no comforter."],
['Ecclesiastes','Ecc','04','002','2',"Wherefore I praised the dead which are already dead more than the living which are yet alive."],
['Ecclesiastes','Ecc','04','003','3',"Yea, better \emph{is} \emph{he} than both they, which hath not yet been, who hath not seen the evil work that is done under the sun."],
['Ecclesiastes','Ecc','04','004','4',"Again, I considered all travail, and every right work, that for this a man is envied of his neighbour. This \emph{is} also vanity and vexation of spirit."],
['Ecclesiastes','Ecc','04','005','5',"The fool foldeth his hands together, and eateth his own flesh."],
['Ecclesiastes','Ecc','04','006','6',"Better \emph{is} an handful \emph{with} quietness, than both the hands full \emph{with} travail and vexation of spirit."],
['Ecclesiastes','Ecc','04','007','7',"Then I returned, and I saw vanity under the sun."],
['Ecclesiastes','Ecc','04','008','8',"There is one \emph{alone}, and \emph{there} \emph{is} not a second; yea, he hath neither child nor brother: yet \emph{is} \emph{there} no end of all his labour; neither is his eye satisfied with riches; neither \emph{saith} \emph{he}, For whom do I labour, and bereave my soul of good? This \emph{is} also vanity, yea, it \emph{is} a sore travail."],
['Ecclesiastes','Ecc','04','009','9',"Two \emph{are} better than one; because they have a good reward for their labour."],
['Ecclesiastes','Ecc','04','010','10',"For if they fall, the one will lift up his fellow: but woe to him \emph{that} \emph{is} alone when he falleth; for \emph{he} \emph{hath} not another to help him up."],
['Ecclesiastes','Ecc','04','011','11',"Again, if two lie together, then they have heat: but how can one be warm \emph{alone}?"],
['Ecclesiastes','Ecc','04','012','12',"And if one prevail against him, two shall withstand him; and a threefold cord is not quickly broken."],
['Ecclesiastes','Ecc','04','013','13',"Better \emph{is} a poor and a wise child than an old and foolish king, who will no more be admonished."],
['Ecclesiastes','Ecc','04','014','14',"For out of prison he cometh to reign; whereas also \emph{he} \emph{that} \emph{is} born in his kingdom becometh poor."],
['Ecclesiastes','Ecc','04','015','15',"I considered all the living which walk under the sun, with the second child that shall stand up in his stead."],
['Ecclesiastes','Ecc','04','016','16',"\emph{There} \emph{is} no end of all the people, \emph{even} of all that have been before them: they also that come after shall not rejoice in him. Surely this also \emph{is} vanity and vexation of spirit."]
]

Ecclesiastes05 = [
['Ecclesiastes','Ecc','05','001','1',"Keep thy foot when thou goest to the house of God, and be more ready to hear, than to give the sacrifice of fools: for they consider not that they do evil."],
['Ecclesiastes','Ecc','05','002','2',"Be not rash with thy mouth, and let not thine heart be hasty to utter \emph{any} thing before God: for God \emph{is} in heaven, and thou upon earth: therefore let thy words be few."],
['Ecclesiastes','Ecc','05','003','3',"For a dream cometh through the multitude of business; and a fool's voice \emph{is} \emph{known} by multitude of words."],
['Ecclesiastes','Ecc','05','004','4',"When thou vowest a vow unto God, defer not to pay it; for \emph{he} \emph{hath} no pleasure in fools: pay that which thou hast vowed."],
['Ecclesiastes','Ecc','05','005','5',"Better \emph{is} \emph{it} that thou shouldest not vow, than that thou shouldest vow and not pay."],
['Ecclesiastes','Ecc','05','006','6',"Suffer not thy mouth to cause thy flesh to sin; neither say thou before the angel, that it \emph{was} an error: wherefore should God be angry at thy voice, and destroy the work of thine hands?"],
['Ecclesiastes','Ecc','05','007','7',"For in the multitude of dreams and many words \emph{there} \emph{are} also \emph{divers} vanities: but fear thou God."],
['Ecclesiastes','Ecc','05','008','8',"If thou seest the oppression of the poor, and violent perverting of judgment and justice in a province, marvel not at the matter: for \emph{he} \emph{that} \emph{is} higher than the highest regardeth; and \emph{there} \emph{be} higher than they."],
['Ecclesiastes','Ecc','05','009','9',"Moreover the profit of the earth is for all: the king \emph{himself} is served by the field."],
['Ecclesiastes','Ecc','05','010','10',"He that loveth silver shall not be satisfied with silver; nor he that loveth abundance with increase: this \emph{is} also vanity."],
['Ecclesiastes','Ecc','05','011','11',"When goods increase, they are increased that eat them: and what good \emph{is} \emph{there} to the owners thereof, saving the beholding \emph{of} \emph{them} with their eyes?"],
['Ecclesiastes','Ecc','05','012','12',"The sleep of a labouring man \emph{is} sweet, whether he eat little or much: but the abundance of the rich will not suffer him to sleep."],
['Ecclesiastes','Ecc','05','013','13',"There is a sore evil \emph{which} I have seen under the sun, \emph{namely}, riches kept for the owners thereof to their hurt."],
['Ecclesiastes','Ecc','05','014','14',"But those riches perish by evil travail: and he begetteth a son, and \emph{there} \emph{is} nothing in his hand."],
['Ecclesiastes','Ecc','05','015','15',"As he came forth of his mother's womb, naked shall he return to go as he came, and shall take nothing of his labour, which he may carry away in his hand."],
['Ecclesiastes','Ecc','05','016','16',"And this also \emph{is} a sore evil, \emph{that} in all points as he came, so shall he go: and what profit hath he that hath laboured for the wind?"],
['Ecclesiastes','Ecc','05','017','17',"All his days also he eateth in darkness, and \emph{he} \emph{hath} much sorrow and wrath with his sickness."],
['Ecclesiastes','Ecc','05','018','18',"Behold \emph{that} which I have seen: \emph{it} \emph{is} good and comely \emph{for} \emph{one} to eat and to drink, and to enjoy the good of all his labour that he taketh under the sun all the days of his life, which God giveth him: for it \emph{is} his portion."],
['Ecclesiastes','Ecc','05','019','19',"Every man also to whom God hath given riches and wealth, and hath given him power to eat thereof, and to take his portion, and to rejoice in his labour; this \emph{is} the gift of God."],
['Ecclesiastes','Ecc','05','020','20',"For he shall not much remember the days of his life; because God answereth \emph{him} in the joy of his heart."]
]

Ecclesiastes06 = [
    ['Ecclesiastes','Ecc','06','001','1',"There is an evil which I have seen under the sun, and it \emph{is} common among men:"],
    ['Ecclesiastes','Ecc','06','002','2',"A man to whom God hath given riches, wealth, and honour, so that he wanteth nothing for his soul of all that he desireth, yet God giveth him not power to eat thereof, but a stranger eateth it: this \emph{is} vanity, and it \emph{is} an evil disease."],
    ['Ecclesiastes','Ecc','06','003','3',"If a man beget an hundred \emph{children}, and live many years, so that the days of his years be many, and his soul be not filled with good, and also \emph{that} he have no burial; I say, \emph{that} an untimely birth \emph{is} better than he."],
    ['Ecclesiastes','Ecc','06','004','4',"For he cometh in with vanity, and departeth in darkness, and his name shall be covered with darkness."],
    ['Ecclesiastes','Ecc','06','005','5',"Moreover he hath not seen the sun, nor known \emph{any} \emph{thing}: this hath more rest than the other."],
    ['Ecclesiastes','Ecc','06','006','6',"Yea, though he live a thousand years twice \emph{told}, yet hath he seen no good: do not all go to one place?"],
    ['Ecclesiastes','Ecc','06','007','7',"All the labour of man \emph{is} for his mouth, and yet the appetite is not filled."],
    ['Ecclesiastes','Ecc','06','008','8',"For what hath the wise more than the fool? what hath the poor, that knoweth to walk before the living?"],
    ['Ecclesiastes','Ecc','06','009','9',"Better \emph{is} the sight of the eyes than the wandering of the desire: this \emph{is} also vanity and vexation of spirit."],
    ['Ecclesiastes','Ecc','06','010','10',"That which hath been is named already, and it is known that it \emph{is} man: neither may he contend with him that is mightier than he."],
    ['Ecclesiastes','Ecc','06','011','11',"Seeing there be many things that increase vanity, what \emph{is} man the better?"],
    ['Ecclesiastes','Ecc','06','012','12',"For who knoweth what \emph{is} good for man in \emph{this} life, all the days of his vain life which he spendeth as a shadow? for who can tell a man what shall be after him under the sun?"]
]

Ecclesiastes07 = [
['Ecclesiastes','Ecc','07','001','1',"A good name \emph{is} better than precious ointment; and the day of death than the day of one's birth."],
['Ecclesiastes','Ecc','07','002','2',"\emph{It} \emph{is} better to go to the house of mourning, than to go to the house of feasting: for that \emph{is} the end of all men; and the living will lay \emph{it} to his heart."],
['Ecclesiastes','Ecc','07','003','3',"Sorrow \emph{is} better than laughter: for by the sadness of the countenance the heart is made better."],
['Ecclesiastes','Ecc','07','004','4',"The heart of the wise \emph{is} in the house of mourning; but the heart of fools \emph{is} in the house of mirth."],
['Ecclesiastes','Ecc','07','005','5',"\emph{It} \emph{is} better to hear the rebuke of the wise, than for a man to hear the song of fools."],
['Ecclesiastes','Ecc','07','006','6',"For as the crackling of thorns under a pot, so \emph{is} the laughter of the fool: this also \emph{is} vanity."],
['Ecclesiastes','Ecc','07','007','7',"Surely oppression maketh a wise man mad; and a gift destroyeth the heart."],
['Ecclesiastes','Ecc','07','008','8',"Better \emph{is} the end of a thing than the beginning thereof: \emph{and} the patient in spirit \emph{is} better than the proud in spirit."],
['Ecclesiastes','Ecc','07','009','9',"Be not hasty in thy spirit to be angry: for anger resteth in the bosom of fools."],
['Ecclesiastes','Ecc','07','010','10',"Say not thou, What is \emph{the} \emph{cause} that the former days were better than these? for thou dost not enquire wisely concerning this."],
['Ecclesiastes','Ecc','07','011','11',"Wisdom \emph{is} good with an inheritance: and \emph{by} \emph{it} \emph{there} \emph{is} profit to them that see the sun."],
['Ecclesiastes','Ecc','07','012','12',"For wisdom \emph{is} a defence, \emph{and} money \emph{is} a defence: but the excellency of knowledge \emph{is,} \emph{that} wisdom giveth life to them that have it."],
['Ecclesiastes','Ecc','07','013','13',"Consider the work of God: for who can make \emph{that} straight, which he hath made crooked?"],
['Ecclesiastes','Ecc','07','014','14',"In the day of prosperity be joyful, but in the day of adversity consider: God also hath set the one over against the other, to the end that man should find nothing after him."],
['Ecclesiastes','Ecc','07','015','15',"All \emph{things} have I seen in the days of my vanity: there is a just \emph{man} that perisheth in his righteousness, and there is a wicked \emph{man} that prolongeth \emph{his} \emph{life} in his wickedness."],
['Ecclesiastes','Ecc','07','016','16',"Be not righteous over much; neither make thyself over wise: why shouldest thou destroy thyself?"],
['Ecclesiastes','Ecc','07','017','17',"Be not over much wicked, neither be thou foolish: why shouldest thou die before thy time?"],
['Ecclesiastes','Ecc','07','018','18',"\emph{It} \emph{is} good that thou shouldest take hold of this; yea, also from this withdraw not thine hand: for he that feareth God shall come forth of them all."],
['Ecclesiastes','Ecc','07','019','19',"Wisdom strengtheneth the wise more than ten mighty \emph{men} which are in the city."],
['Ecclesiastes','Ecc','07','020','20',"For \emph{there} \emph{is} not a just man upon earth, that doeth good, and sinneth not."],
['Ecclesiastes','Ecc','07','021','21',"Also take no heed unto all words that are spoken; lest thou hear thy servant curse thee:"],
['Ecclesiastes','Ecc','07','022','22',"For oftentimes also thine own heart knoweth that thou thyself likewise hast cursed others."],
['Ecclesiastes','Ecc','07','023','23',"All this have I proved by wisdom: I said, I will be wise; but it \emph{was} far from me."],
['Ecclesiastes','Ecc','07','024','24',"That which is far off, and exceeding deep, who can find it out?"],
['Ecclesiastes','Ecc','07','025','25',"I applied mine heart to know, and to search, and to seek out wisdom, and the reason \emph{of} \emph{things}, and to know the wickedness of folly, even of foolishness \emph{and} madness:"],
['Ecclesiastes','Ecc','07','026','26',"And I find more bitter than death the woman, whose heart \emph{is} snares and nets, \emph{and} her hands \emph{as} bands: whoso pleaseth God shall escape from her; but the sinner shall be taken by her."],
['Ecclesiastes','Ecc','07','027','27',"Behold, this have I found, saith the preacher, \emph{counting} one by one, to find out the account:"],
['Ecclesiastes','Ecc','07','028','28',"Which yet my soul seeketh, but I find not: one man among a thousand have I found; but a woman among all those have I not found."],
['Ecclesiastes','Ecc','07','029','29',"Lo, this only have I found, that God hath made man upright; but they have sought out many inventions."]
]

Ecclesiastes08 = [
['Ecclesiastes','Ecc','08','001','1',"Who \emph{is} as the wise \emph{man}? and who knoweth the interpretation of a thing? a man's wisdom maketh his face to shine, and the boldness of his face shall be changed."],
['Ecclesiastes','Ecc','08','002','2',"I \emph{counsel} \emph{thee} to keep the king's commandment, and \emph{that} in regard of the oath of God."],
['Ecclesiastes','Ecc','08','003','3',"Be not hasty to go out of his sight: stand not in an evil thing; for he doeth whatsoever pleaseth him."],
['Ecclesiastes','Ecc','08','004','4',"Where the word of a king \emph{is,} \emph{there} \emph{is} power: and who may say unto him, What doest thou?"],
['Ecclesiastes','Ecc','08','005','5',"Whoso keepeth the commandment shall feel no evil thing: and a wise man's heart discerneth both time and judgment."],
['Ecclesiastes','Ecc','08','006','6',"Because to every purpose there is time and judgment, therefore the misery of man \emph{is} great upon him."],
['Ecclesiastes','Ecc','08','007','7',"For he knoweth not that which shall be: for who can tell him when it shall be?"],
['Ecclesiastes','Ecc','08','008','8',"\emph{There} \emph{is} no man that hath power over the spirit to retain the spirit; neither \emph{hath} \emph{he} power in the day of death: and \emph{there} \emph{is} no discharge in \emph{that} war; neither shall wickedness deliver those that are given to it."],
['Ecclesiastes','Ecc','08','009','9',"All this have I seen, and applied my heart unto every work that is done under the sun: \emph{there} \emph{is} a time wherein one man ruleth over another to his own hurt."],
['Ecclesiastes','Ecc','08','010','10',"And so I saw the wicked buried, who had come and gone from the place of the holy, and they were forgotten in the city where they had so done: this \emph{is} also vanity."],
['Ecclesiastes','Ecc','08','011','11',"Because sentence against an evil work is not executed speedily, therefore the heart of the sons of men is fully set in them to do evil."],
['Ecclesiastes','Ecc','08','012','12',"Though a sinner do evil an hundred times, and his \emph{days} be prolonged, yet surely I know that it shall be well with them that fear God, which fear before him:"],
['Ecclesiastes','Ecc','08','013','13',"But it shall not be well with the wicked, neither shall he prolong \emph{his} days, \emph{which} \emph{are} as a shadow; because he feareth not before God."],
['Ecclesiastes','Ecc','08','014','14',"There is a vanity which is done upon the earth; that there be just \emph{men}, unto whom it happeneth according to the work of the wicked; again, there be wicked \emph{men}, to whom it happeneth according to the work of the righteous: I said that this also \emph{is} vanity."],
['Ecclesiastes','Ecc','08','015','15',"Then I commended mirth, because a man hath no better thing under the sun, than to eat, and to drink, and to be merry: for that shall abide with him of his labour the days of his life, which God giveth him under the sun."],
['Ecclesiastes','Ecc','08','016','16',"When I applied mine heart to know wisdom, and to see the business that is done upon the earth: (for also \emph{there} \emph{is} \emph{that} neither day nor night seeth sleep with his eyes:)"],
['Ecclesiastes','Ecc','08','017','17',"Then I beheld all the work of God, that a man cannot find out the work that is done under the sun: because though a man labour to seek \emph{it} out, yet he shall not find \emph{it}; yea further; though a wise \emph{man} think to know \emph{it}, yet shall he not be able to find \emph{it}."]
]


Ecclesiastes09 = [
['Ecclesiastes','Ecc','09','001','1',"For all this I considered in my heart even to declare all this, that the righteous, and the wise, and their works, \emph{are} in the hand of God: no man knoweth either love or hatred \emph{by} all \emph{that} \emph{is} before them."],
['Ecclesiastes','Ecc','09','002','2',"All \emph{things} \emph{come} alike to all: \emph{there} \emph{is} one event to the righteous, and to the wicked; to the good and to the clean, and to the unclean; to him that sacrificeth, and to him that sacrificeth not: as \emph{is} the good, so \emph{is} the sinner; \emph{and} he that sweareth, as \emph{he} that feareth an oath."],
['Ecclesiastes','Ecc','09','003','3',"This \emph{is} an evil among all \emph{things} that are done under the sun, that \emph{there} \emph{is} one event unto all: yea, also the heart of the sons of men is full of evil, and madness \emph{is} in their heart while they live, and after that \emph{they} \emph{go} to the dead."],
['Ecclesiastes','Ecc','09','004','4',"For to him that is joined to all the living there is hope: for a living dog is better than a dead lion."],
['Ecclesiastes','Ecc','09','005','5',"For the living know that they shall die: but the dead know not any thing, neither have they any more a reward; for the memory of them is forgotten."],
['Ecclesiastes','Ecc','09','006','6',"Also their love, and their hatred, and their envy, is now perished; neither have they any more a portion for ever in any \emph{thing} that is done under the sun."],
['Ecclesiastes','Ecc','09','007','7',"Go thy way, eat thy bread with joy, and drink thy wine with a merry heart; for God now accepteth thy works."],
['Ecclesiastes','Ecc','09','008','8',"Let thy garments be always white; and let thy head lack no ointment."],
['Ecclesiastes','Ecc','09','009','9',"Live joyfully with the wife whom thou lovest all the days of the life of thy vanity, which he hath given thee under the sun, all the days of thy vanity: for that \emph{is} thy portion in \emph{this} life, and in thy labour which thou takest under the sun."],
['Ecclesiastes','Ecc','09','010','10',"Whatsoever thy hand findeth to do, do \emph{it} with thy might; for \emph{there} \emph{is} no work, nor device, nor knowledge, nor wisdom, in the grave, whither thou goest."],
['Ecclesiastes','Ecc','09','011','11',"I returned, and saw under the sun, that the race \emph{is} not to the swift, nor the battle to the strong, neither yet bread to the wise, nor yet riches to men of understanding, nor yet favour to men of skill; but time and chance happeneth to them all."],
['Ecclesiastes','Ecc','09','012','12',"For man also knoweth not his time: as the fishes that are taken in an evil net, and as the birds that are caught in the snare; so \emph{are} the sons of men snared in an evil time, when it falleth suddenly upon them."],
['Ecclesiastes','Ecc','09','013','13',"This wisdom have I seen also under the sun, and it \emph{seemed} great unto me:"],
['Ecclesiastes','Ecc','09','014','14',"\emph{There} \emph{was} a little city, and few men within it; and there came a great king against it, and besieged it, and built great bulwarks against it:"],
['Ecclesiastes','Ecc','09','015','15',"Now there was found in it a poor wise man, and he by his wisdom delivered the city; yet no man remembered that same poor man."],
['Ecclesiastes','Ecc','09','016','16',"Then said I, Wisdom \emph{is} better than strength: nevertheless the poor man's wisdom \emph{is} despised, and his words are not heard."],
['Ecclesiastes','Ecc','09','017','17',"The words of wise \emph{men} \emph{are} heard in quiet more than the cry of him that ruleth among fools."],
['Ecclesiastes','Ecc','09','018','18',"Wisdom \emph{is} better than weapons of war: but one sinner destroyeth much good."]
]

Ecclesiastes10 = [
['Ecclesiastes','Ecc','10','001','1',"Dead flies cause the ointment of the apothecary to send forth a stinking savour: \emph{so} \emph{doth} a little folly him that is in reputation for wisdom \emph{and} honour."],
['Ecclesiastes','Ecc','10','002','2',"A wise man's heart \emph{is} at his right hand; but a fool's heart at his left."],
['Ecclesiastes','Ecc','10','003','3',"Yea also, when he that is a fool walketh by the way, his wisdom faileth \emph{him}, and he saith to every one \emph{that} he \emph{is} a fool."],
['Ecclesiastes','Ecc','10','004','4',"If the spirit of the ruler rise up against thee, leave not thy place; for yielding pacifieth great offences."],
['Ecclesiastes','Ecc','10','005','5',"There is an evil \emph{which} I have seen under the sun, as an error \emph{which} proceedeth from the ruler:"],
['Ecclesiastes','Ecc','10','006','6',"Folly is set in great dignity, and the rich sit in low place."],
['Ecclesiastes','Ecc','10','007','7',"I have seen servants upon horses, and princes walking as servants upon the earth."],
['Ecclesiastes','Ecc','10','008','8',"He that diggeth a pit shall fall into it; and whoso breaketh an hedge, a serpent shall bite him."],
['Ecclesiastes','Ecc','10','009','9',"Whoso removeth stones shall be hurt therewith; \emph{and} he that cleaveth wood shall be endangered thereby."],
['Ecclesiastes','Ecc','10','010','10',"If the iron be blunt, and he do not whet the edge, then must he put to more strength: but wisdom \emph{is} profitable to direct."],
['Ecclesiastes','Ecc','10','011','11',"Surely the serpent will bite without enchantment; and a babbler is no better."],
['Ecclesiastes','Ecc','10','012','12',"The words of a wise man's mouth \emph{are} gracious; but the lips of a fool will swallow up himself."],
['Ecclesiastes','Ecc','10','013','13',"The beginning of the words of his mouth \emph{is} foolishness: and the end of his talk \emph{is} mischievous madness."],
['Ecclesiastes','Ecc','10','014','14',"A fool also is full of words: a man cannot tell what shall be; and what shall be after him, who can tell him?"],
['Ecclesiastes','Ecc','10','015','15',"The labour of the foolish wearieth every one of them, because he knoweth not how to go to the city."],
['Ecclesiastes','Ecc','10','016','16',"Woe to thee, O land, when thy king \emph{is} a child, and thy princes eat in the morning!"],
['Ecclesiastes','Ecc','10','017','17',"Blessed \emph{art} thou, O land, when thy king \emph{is} the son of nobles, and thy princes eat in due season, for strength, and not for drunkenness!"],
['Ecclesiastes','Ecc','10','018','18',"By much slothfulness the building decayeth; and through idleness of the hands the house droppeth through."],
['Ecclesiastes','Ecc','10','019','19',"A feast is made for laughter, and wine maketh merry: but money answereth all \emph{things}."],
['Ecclesiastes','Ecc','10','020','20',"Curse not the king, no not in thy thought; and curse not the rich in thy bedchamber: for a bird of the air shall carry the voice, and that which hath wings shall tell the matter."]
]

Ecclesiastes11 = [
['Ecclesiastes','Ecc','11','001','1',"Cast thy bread upon the waters: for thou shalt find it after many days."],
['Ecclesiastes','Ecc','11','002','2',"Give a portion to seven, and also to eight; for thou knowest not what evil shall be upon the earth."],
['Ecclesiastes','Ecc','11','003','3',"If the clouds be full of rain, they empty \emph{themselves} upon the earth: and if the tree fall toward the south, or toward the north, in the place where the tree falleth, there it shall be."],
['Ecclesiastes','Ecc','11','004','4',"He that observeth the wind shall not sow; and he that regardeth the clouds shall not reap."],
['Ecclesiastes','Ecc','11','005','5',"As thou knowest not what \emph{is} the way of the spirit, \emph{nor} how the bones \emph{do} \emph{grow} in the womb of her that is with child: even so thou knowest not the works of God who maketh all."],
['Ecclesiastes','Ecc','11','006','6',"In the morning sow thy seed, and in the evening withhold not thine hand: for thou knowest not whether shall prosper, either this or that, or whether they both \emph{shall} \emph{be} alike good."],
['Ecclesiastes','Ecc','11','007','7',"Truly the light \emph{is} sweet, and a pleasant \emph{thing} \emph{it} \emph{is} for the eyes to behold the sun:"],
['Ecclesiastes','Ecc','11','008','8',"But if a man live many years, \emph{and} rejoice in them all; yet let him remember the days of darkness; for they shall be many. All that cometh \emph{is} vanity."],
['Ecclesiastes','Ecc','11','009','9',"Rejoice, O young man, in thy youth; and let thy heart cheer thee in the days of thy youth, and walk in the ways of thine heart, and in the sight of thine eyes: but know thou, that for all these \emph{things} God will bring thee into judgment."],
['Ecclesiastes','Ecc','11','010','10',"Therefore remove sorrow from thy heart, and put away evil from thy flesh: for childhood and youth \emph{are} vanity."]
]

Ecclesiastes12 = [
['Ecclesiastes','Ecc','12','001','1',"Remember now thy Creator in the days of thy youth, while the evil days come not, nor the years draw nigh, when thou shalt say, I have no pleasure in them;"],
['Ecclesiastes','Ecc','12','002','2',"While the sun, or the light, or the moon, or the stars, be not darkened, nor the clouds return after the rain:"],
['Ecclesiastes','Ecc','12','003','3',"In the day when the keepers of the house shall tremble, and the strong men shall bow themselves, and the grinders cease because they are few, and those that look out of the windows be darkened,"],
['Ecclesiastes','Ecc','12','004','4',"And the doors shall be shut in the streets, when the sound of the grinding is low, and he shall rise up at the voice of the bird, and all the daughters of musick shall be brought low;"],
['Ecclesiastes','Ecc','12','005','5',"Also \emph{when} they shall be afraid of \emph{that} \emph{which} \emph{is} high, and fears \emph{shall} \emph{be} in the way, and the almond tree shall flourish, and the grasshopper shall be a burden, and desire shall fail: because man goeth to his long home, and the mourners go about the streets:"],
['Ecclesiastes','Ecc','12','006','6',"Or ever the silver cord be loosed, or the golden bowl be broken, or the pitcher be broken at the fountain, or the wheel broken at the cistern."],
['Ecclesiastes','Ecc','12','007','7',"Then shall the dust return to the earth as it was: and the spirit shall return unto God who gave it."],
['Ecclesiastes','Ecc','12','008','8',"Vanity of vanities, saith the preacher; all \emph{is} vanity."],
['Ecclesiastes','Ecc','12','009','9',"And moreover, because the preacher was wise, he still taught the people knowledge; yea, he gave good heed, and sought out, \emph{and} set in order many proverbs."],
['Ecclesiastes','Ecc','12','010','10',"The preacher sought to find out acceptable words: and \emph{that} \emph{which} \emph{was} written \emph{was} upright, \emph{even} words of truth."],
['Ecclesiastes','Ecc','12','011','11',"The words of the wise \emph{are} as goads, and as nails fastened \emph{by} the masters of assemblies, \emph{which} are given from one shepherd."],
['Ecclesiastes','Ecc','12','012','12',"And further, by these, my son, be admonished: of making many books \emph{there} \emph{is} no end; and much study \emph{is} a weariness of the flesh."],
['Ecclesiastes','Ecc','12','013','13',"Let us hear the conclusion of the whole matter: Fear God, and keep his commandments: for this \emph{is} the whole \emph{duty} of man."],
['Ecclesiastes','Ecc','12','014','14',"For God shall bring every work into judgment, with every secret thing, whether \emph{it} \emph{be} good, or whether \emph{it} \emph{be} evil."]
]

EccleiastesAll = [
    ['Ecclesiastes','Ecc','01','001','1',"The words of the Preacher, the son of David, king in Jerusalem."],
    ['Ecclesiastes','Ecc','01','002','2',"Vanity of vanities, saith the Preacher, vanity of vanities; all \emph{is} vanity."],
    ['Ecclesiastes','Ecc','01','003','3',"What profit hath a man of all his labour which he taketh under the sun?"],
    ['Ecclesiastes','Ecc','01','004','4',"\emph{One} generation passeth away, and \emph{another} generation cometh: but the earth abideth for ever."],
    ['Ecclesiastes','Ecc','01','005','5',"The sun also ariseth, and the sun goeth down, and hasteth to his place where he arose."],
    ['Ecclesiastes','Ecc','01','006','6',"The wind goeth toward the south, and turneth about unto the north; it whirleth about continually, and the wind returneth again according to his circuits."],
    ['Ecclesiastes','Ecc','01','007','7',"All the rivers run into the sea; yet the sea \emph{is} not full; unto the place from whence the rivers come, thither they return again."],
    ['Ecclesiastes','Ecc','01','008','8',"All things \emph{are} full of labour; man cannot utter \emph{it}: the eye is not satisfied with seeing, nor the ear filled with hearing."],
    ['Ecclesiastes','Ecc','01','009','9',"The thing that hath been, it \emph{is} \emph{that} which shall be; and that which is done \emph{is} that which shall be done: and \emph{there} \emph{is} no new \emph{thing} under the sun."],
    ['Ecclesiastes','Ecc','01','010','10',"Is there \emph{any} thing whereof it may be said, See, this \emph{is} new? it hath been already of old time, which was before us."],
    ['Ecclesiastes','Ecc','01','011','11',"\emph{There} \emph{is} no remembrance of former \emph{things}; neither shall there be \emph{any} remembrance of \emph{things} that are to come with \emph{those} that shall come after."],
    ['Ecclesiastes','Ecc','01','012','12',"I the Preacher was king over Israel in Jerusalem."],
    ['Ecclesiastes','Ecc','01','013','13',"And I gave my heart to seek and search out by wisdom concerning all \emph{things} that are done under heaven: this sore travail hath God given to the sons of man to be exercised therewith."],
    ['Ecclesiastes','Ecc','01','014','14',"I have seen all the works that are done under the sun; and, behold, all \emph{is} vanity and vexation of spirit."],
    ['Ecclesiastes','Ecc','01','015','15',"\emph{That} \emph{which} \emph{is} crooked cannot be made straight: and that which is wanting cannot be numbered."],
    ['Ecclesiastes','Ecc','01','016','16',"I communed with mine own heart, saying, Lo, I am come to great estate, and have gotten more wisdom than all \emph{they} that have been before me in Jerusalem: yea, my heart had great experience of wisdom and knowledge."],
    ['Ecclesiastes','Ecc','01','017','17',"And I gave my heart to know wisdom, and to know madness and folly: I perceived that this also is vexation of spirit."],
    ['Ecclesiastes','Ecc','01','018','18',"For in much wisdom \emph{is} much grief: and he that increaseth knowledge increaseth sorrow."],
['Ecclesiastes','Ecc','02','001','1',"I said in mine heart, Go to now, I will prove thee with mirth, therefore enjoy pleasure: and, behold, this also \emph{is} vanity."],
['Ecclesiastes','Ecc','02','002','2',"I said of laughter, \emph{It} \emph{is} mad: and of mirth, What doeth it?"],
['Ecclesiastes','Ecc','02','003','3',"I sought in mine heart to give myself unto wine, yet acquainting mine heart with wisdom; and to lay hold on folly, till I might see what \emph{was} that good for the sons of men, which they should do under the heaven all the days of their life."],
['Ecclesiastes','Ecc','02','004','4',"I made me great works; I builded me houses; I planted me vineyards:"],
['Ecclesiastes','Ecc','02','005','5',"I made me gardens and orchards, and I planted trees in them of all \emph{kind} \emph{of} fruits:"],
['Ecclesiastes','Ecc','02','006','6',"I made me pools of water, to water therewith the wood that bringeth forth trees:"],
['Ecclesiastes','Ecc','02','007','7',"I got \emph{me} servants and maidens, and had servants born in my house; also I had great possessions of great and small cattle above all that were in Jerusalem before me:"],
['Ecclesiastes','Ecc','02','008','8',"I gathered me also silver and gold, and the peculiar treasure of kings and of the provinces: I gat me men singers and women singers, and the delights of the sons of men, \emph{as} musical instruments, and that of all sorts."],
['Ecclesiastes','Ecc','02','009','9',"So I was great, and increased more than all that were before me in Jerusalem: also my wisdom remained with me."],
['Ecclesiastes','Ecc','02','010','10',"And whatsoever mine eyes desired I kept not from them, I withheld not my heart from any joy; for my heart rejoiced in all my labour: and this was my portion of all my labour."],
['Ecclesiastes','Ecc','02','011','11',"Then I looked on all the works that my hands had wrought, and on the labour that I had laboured to do: and, behold, all \emph{was} vanity and vexation of spirit, and \emph{there} \emph{was} no profit under the sun."],
['Ecclesiastes','Ecc','02','012','12',"And I turned myself to behold wisdom, and madness, and folly: for what \emph{can} the man \emph{do} that cometh after the king? \emph{even} that which hath been already done."],
['Ecclesiastes','Ecc','02','013','13',"Then I saw that wisdom excelleth folly, as far as light excelleth darkness."],
['Ecclesiastes','Ecc','02','014','14',"The wise man's eyes \emph{are} in his head; but the fool walketh in darkness: and I myself perceived also that one event happeneth to them all."],
['Ecclesiastes','Ecc','02','015','15',"Then said I in my heart, As it happeneth to the fool, so it happeneth even to me; and why was I then more wise? Then I said in my heart, that this also \emph{is} vanity."],
['Ecclesiastes','Ecc','02','016','16',"For \emph{there} \emph{is} no remembrance of the wise more than of the fool for ever; seeing that which now \emph{is} in the days to come shall all be forgotten. And how dieth the wise \emph{man}? as the fool."],
['Ecclesiastes','Ecc','02','017','17',"Therefore I hated life; because the work that is wrought under the sun \emph{is} grievous unto me: for all \emph{is} vanity and vexation of spirit."],
['Ecclesiastes','Ecc','02','018','18',"Yea, I hated all my labour which I had taken under the sun: because I should leave it unto the man that shall be after me."],
['Ecclesiastes','Ecc','02','019','19',"And who knoweth whether he shall be a wise \emph{man} or a fool? yet shall he have rule over all my labour wherein I have laboured, and wherein I have shewed myself wise under the sun. This \emph{is} also vanity."],
['Ecclesiastes','Ecc','02','020','20',"Therefore I went about to cause my heart to despair of all the labour which I took under the sun."],
['Ecclesiastes','Ecc','02','021','21',"For there is a man whose labour \emph{is} in wisdom, and in knowledge, and in equity; yet to a man that hath not laboured therein shall he leave it \emph{for} his portion. This also \emph{is} vanity and a great evil."],
['Ecclesiastes','Ecc','02','022','22',"For what hath man of all his labour, and of the vexation of his heart, wherein he hath laboured under the sun?"],
['Ecclesiastes','Ecc','02','023','23',"For all his days \emph{are} sorrows, and his travail grief; yea, his heart taketh not rest in the night. This is also vanity."],
['Ecclesiastes','Ecc','02','024','24',"\emph{There} \emph{is} nothing better for a man, \emph{than} that he should eat and drink, and \emph{that} he should make his soul enjoy good in his labour. This also I saw, that it \emph{was} from the hand of God."],
['Ecclesiastes','Ecc','02','025','25',"For who can eat, or who else can hasten \emph{hereunto}, more than I?"],
['Ecclesiastes','Ecc','02','026','26',"For \emph{God} giveth to a man that \emph{is} good in his sight wisdom, and knowledge, and joy: but to the sinner he giveth travail, to gather and to heap up, that he may give to \emph{him} \emph{that} \emph{is} good before God. This also \emph{is} vanity and vexation of spirit."],
['Ecclesiastes','Ecc','03','001','1',"To every \emph{thing} \emph{there} \emph{is} a season, and a time to every purpose under the heaven:"],
['Ecclesiastes','Ecc','03','002','2',"A time to be born, and a time to die; a time to plant, and a time to pluck up \emph{that} \emph{which} \emph{is} planted;"],
['Ecclesiastes','Ecc','03','003','3',"A time to kill, and a time to heal; a time to break down, and a time to build up;"],
['Ecclesiastes','Ecc','03','004','4',"A time to weep, and a time to laugh; a time to mourn, and a time to dance;"],
['Ecclesiastes','Ecc','03','005','5',"A time to cast away stones, and a time to gather stones together; a time to embrace, and a time to refrain from embracing;"],
['Ecclesiastes','Ecc','03','006','6',"A time to get, and a time to lose; a time to keep, and a time to cast away;"],
['Ecclesiastes','Ecc','03','007','7',"A time to rend, and a time to sew; a time to keep silence, and a time to speak;"],
['Ecclesiastes','Ecc','03','008','8',"A time to love, and a time to hate; a time of war, and a time of peace."],
['Ecclesiastes','Ecc','03','009','9',"What profit hath he that worketh in that wherein he laboureth?"],
['Ecclesiastes','Ecc','03','010','10',"I have seen the travail, which God hath given to the sons of men to be exercised in it."],
['Ecclesiastes','Ecc','03','011','11',"He hath made every \emph{thing} beautiful in his time: also he hath set the world in their heart, so that no man can find out the work that God maketh from the beginning to the end."],
['Ecclesiastes','Ecc','03','012','12',"I know that \emph{there} \emph{is} no good in them, but for \emph{a} \emph{man} to rejoice, and to do good in his life."],
['Ecclesiastes','Ecc','03','013','13',"And also that every man should eat and drink, and enjoy the good of all his labour, it \emph{is} the gift of God."],
['Ecclesiastes','Ecc','03','014','14',"I know that, whatsoever God doeth, it shall be for ever: nothing can be put to it, nor any thing taken from it: and God doeth \emph{it}, that \emph{men} should fear before him."],
['Ecclesiastes','Ecc','03','015','15',"That which hath been is now; and that which is to be hath already been; and God requireth that which is past."],
['Ecclesiastes','Ecc','03','016','16',"And moreover I saw under the sun the place of judgment, \emph{that} wickedness \emph{was} there; and the place of righteousness, \emph{that} iniquity \emph{was} there."],
['Ecclesiastes','Ecc','03','017','17',"I said in mine heart, God shall judge the righteous and the wicked: for \emph{there} \emph{is} a time there for every purpose and for every work."],
['Ecclesiastes','Ecc','03','018','18',"I said in mine heart concerning the estate of the sons of men, that God might manifest them, and that they might see that they themselves are beasts."],
['Ecclesiastes','Ecc','03','019','19',"For that which befalleth the sons of men befalleth beasts; even one thing befalleth them: as the one dieth, so dieth the other; yea, they have all one breath; so that a man hath no preeminence above a beast: for all \emph{is} vanity."],
['Ecclesiastes','Ecc','03','020','20',"All go unto one place; all are of the dust, and all turn to dust again."],
['Ecclesiastes','Ecc','03','021','21',"Who knoweth the spirit of man that goeth upward, and the spirit of the beast that goeth downward to the earth?"],
['Ecclesiastes','Ecc','03','022','22',"Wherefore I perceive that \emph{there} \emph{is} nothing better, than that a man should rejoice in his own works; for that \emph{is} his portion: for who shall bring him to see what shall be after him?"],
['Ecclesiastes','Ecc','04','001','1',"So I returned, and considered all the oppressions that are done under the sun: and behold the tears of \emph{such} \emph{as} \emph{were} oppressed, and they had no comforter; and on the side of their oppressors \emph{there} \emph{was} power; but they had no comforter."],
['Ecclesiastes','Ecc','04','002','2',"Wherefore I praised the dead which are already dead more than the living which are yet alive."],
['Ecclesiastes','Ecc','04','003','3',"Yea, better \emph{is} \emph{he} than both they, which hath not yet been, who hath not seen the evil work that is done under the sun."],
['Ecclesiastes','Ecc','04','004','4',"Again, I considered all travail, and every right work, that for this a man is envied of his neighbour. This \emph{is} also vanity and vexation of spirit."],
['Ecclesiastes','Ecc','04','005','5',"The fool foldeth his hands together, and eateth his own flesh."],
['Ecclesiastes','Ecc','04','006','6',"Better \emph{is} an handful \emph{with} quietness, than both the hands full \emph{with} travail and vexation of spirit."],
['Ecclesiastes','Ecc','04','007','7',"Then I returned, and I saw vanity under the sun."],
['Ecclesiastes','Ecc','04','008','8',"There is one \emph{alone}, and \emph{there} \emph{is} not a second; yea, he hath neither child nor brother: yet \emph{is} \emph{there} no end of all his labour; neither is his eye satisfied with riches; neither \emph{saith} \emph{he}, For whom do I labour, and bereave my soul of good? This \emph{is} also vanity, yea, it \emph{is} a sore travail."],
['Ecclesiastes','Ecc','04','009','9',"Two \emph{are} better than one; because they have a good reward for their labour."],
['Ecclesiastes','Ecc','04','010','10',"For if they fall, the one will lift up his fellow: but woe to him \emph{that} \emph{is} alone when he falleth; for \emph{he} \emph{hath} not another to help him up."],
['Ecclesiastes','Ecc','04','011','11',"Again, if two lie together, then they have heat: but how can one be warm \emph{alone}?"],
['Ecclesiastes','Ecc','04','012','12',"And if one prevail against him, two shall withstand him; and a threefold cord is not quickly broken."],
['Ecclesiastes','Ecc','04','013','13',"Better \emph{is} a poor and a wise child than an old and foolish king, who will no more be admonished."],
['Ecclesiastes','Ecc','04','014','14',"For out of prison he cometh to reign; whereas also \emph{he} \emph{that} \emph{is} born in his kingdom becometh poor."],
['Ecclesiastes','Ecc','04','015','15',"I considered all the living which walk under the sun, with the second child that shall stand up in his stead."],
['Ecclesiastes','Ecc','04','016','16',"\emph{There} \emph{is} no end of all the people, \emph{even} of all that have been before them: they also that come after shall not rejoice in him. Surely this also \emph{is} vanity and vexation of spirit."],
['Ecclesiastes','Ecc','05','001','1',"Keep thy foot when thou goest to the house of God, and be more ready to hear, than to give the sacrifice of fools: for they consider not that they do evil."],
['Ecclesiastes','Ecc','05','002','2',"Be not rash with thy mouth, and let not thine heart be hasty to utter \emph{any} thing before God: for God \emph{is} in heaven, and thou upon earth: therefore let thy words be few."],
['Ecclesiastes','Ecc','05','003','3',"For a dream cometh through the multitude of business; and a fool's voice \emph{is} \emph{known} by multitude of words."],
['Ecclesiastes','Ecc','05','004','4',"When thou vowest a vow unto God, defer not to pay it; for \emph{he} \emph{hath} no pleasure in fools: pay that which thou hast vowed."],
['Ecclesiastes','Ecc','05','005','5',"Better \emph{is} \emph{it} that thou shouldest not vow, than that thou shouldest vow and not pay."],
['Ecclesiastes','Ecc','05','006','6',"Suffer not thy mouth to cause thy flesh to sin; neither say thou before the angel, that it \emph{was} an error: wherefore should God be angry at thy voice, and destroy the work of thine hands?"],
['Ecclesiastes','Ecc','05','007','7',"For in the multitude of dreams and many words \emph{there} \emph{are} also \emph{divers} vanities: but fear thou God."],
['Ecclesiastes','Ecc','05','008','8',"If thou seest the oppression of the poor, and violent perverting of judgment and justice in a province, marvel not at the matter: for \emph{he} \emph{that} \emph{is} higher than the highest regardeth; and \emph{there} \emph{be} higher than they."],
['Ecclesiastes','Ecc','05','009','9',"Moreover the profit of the earth is for all: the king \emph{himself} is served by the field."],
['Ecclesiastes','Ecc','05','010','10',"He that loveth silver shall not be satisfied with silver; nor he that loveth abundance with increase: this \emph{is} also vanity."],
['Ecclesiastes','Ecc','05','011','11',"When goods increase, they are increased that eat them: and what good \emph{is} \emph{there} to the owners thereof, saving the beholding \emph{of} \emph{them} with their eyes?"],
['Ecclesiastes','Ecc','05','012','12',"The sleep of a labouring man \emph{is} sweet, whether he eat little or much: but the abundance of the rich will not suffer him to sleep."],
['Ecclesiastes','Ecc','05','013','13',"There is a sore evil \emph{which} I have seen under the sun, \emph{namely}, riches kept for the owners thereof to their hurt."],
['Ecclesiastes','Ecc','05','014','14',"But those riches perish by evil travail: and he begetteth a son, and \emph{there} \emph{is} nothing in his hand."],
['Ecclesiastes','Ecc','05','015','15',"As he came forth of his mother's womb, naked shall he return to go as he came, and shall take nothing of his labour, which he may carry away in his hand."],
['Ecclesiastes','Ecc','05','016','16',"And this also \emph{is} a sore evil, \emph{that} in all points as he came, so shall he go: and what profit hath he that hath laboured for the wind?"],
['Ecclesiastes','Ecc','05','017','17',"All his days also he eateth in darkness, and \emph{he} \emph{hath} much sorrow and wrath with his sickness."],
['Ecclesiastes','Ecc','05','018','18',"Behold \emph{that} which I have seen: \emph{it} \emph{is} good and comely \emph{for} \emph{one} to eat and to drink, and to enjoy the good of all his labour that he taketh under the sun all the days of his life, which God giveth him: for it \emph{is} his portion."],
['Ecclesiastes','Ecc','05','019','19',"Every man also to whom God hath given riches and wealth, and hath given him power to eat thereof, and to take his portion, and to rejoice in his labour; this \emph{is} the gift of God."],
['Ecclesiastes','Ecc','05','020','20',"For he shall not much remember the days of his life; because God answereth \emph{him} in the joy of his heart."],
    ['Ecclesiastes','Ecc','06','001','1',"There is an evil which I have seen under the sun, and it \emph{is} common among men:"],
    ['Ecclesiastes','Ecc','06','002','2',"A man to whom God hath given riches, wealth, and honour, so that he wanteth nothing for his soul of all that he desireth, yet God giveth him not power to eat thereof, but a stranger eateth it: this \emph{is} vanity, and it \emph{is} an evil disease."],
    ['Ecclesiastes','Ecc','06','003','3',"If a man beget an hundred \emph{children}, and live many years, so that the days of his years be many, and his soul be not filled with good, and also \emph{that} he have no burial; I say, \emph{that} an untimely birth \emph{is} better than he."],
    ['Ecclesiastes','Ecc','06','004','4',"For he cometh in with vanity, and departeth in darkness, and his name shall be covered with darkness."],
    ['Ecclesiastes','Ecc','06','005','5',"Moreover he hath not seen the sun, nor known \emph{any} \emph{thing}: this hath more rest than the other."],
    ['Ecclesiastes','Ecc','06','006','6',"Yea, though he live a thousand years twice \emph{told}, yet hath he seen no good: do not all go to one place?"],
    ['Ecclesiastes','Ecc','06','007','7',"All the labour of man \emph{is} for his mouth, and yet the appetite is not filled."],
    ['Ecclesiastes','Ecc','06','008','8',"For what hath the wise more than the fool? what hath the poor, that knoweth to walk before the living?"],
    ['Ecclesiastes','Ecc','06','009','9',"Better \emph{is} the sight of the eyes than the wandering of the desire: this \emph{is} also vanity and vexation of spirit."],
    ['Ecclesiastes','Ecc','06','010','10',"That which hath been is named already, and it is known that it \emph{is} man: neither may he contend with him that is mightier than he."],
    ['Ecclesiastes','Ecc','06','011','11',"Seeing there be many things that increase vanity, what \emph{is} man the better?"],
    ['Ecclesiastes','Ecc','06','012','12',"For who knoweth what \emph{is} good for man in \emph{this} life, all the days of his vain life which he spendeth as a shadow? for who can tell a man what shall be after him under the sun?"],
['Ecclesiastes','Ecc','07','001','1',"A good name \emph{is} better than precious ointment; and the day of death than the day of one's birth."],
['Ecclesiastes','Ecc','07','002','2',"\emph{It} \emph{is} better to go to the house of mourning, than to go to the house of feasting: for that \emph{is} the end of all men; and the living will lay \emph{it} to his heart."],
['Ecclesiastes','Ecc','07','003','3',"Sorrow \emph{is} better than laughter: for by the sadness of the countenance the heart is made better."],
['Ecclesiastes','Ecc','07','004','4',"The heart of the wise \emph{is} in the house of mourning; but the heart of fools \emph{is} in the house of mirth."],
['Ecclesiastes','Ecc','07','005','5',"\emph{It} \emph{is} better to hear the rebuke of the wise, than for a man to hear the song of fools."],
['Ecclesiastes','Ecc','07','006','6',"For as the crackling of thorns under a pot, so \emph{is} the laughter of the fool: this also \emph{is} vanity."],
['Ecclesiastes','Ecc','07','007','7',"Surely oppression maketh a wise man mad; and a gift destroyeth the heart."],
['Ecclesiastes','Ecc','07','008','8',"Better \emph{is} the end of a thing than the beginning thereof: \emph{and} the patient in spirit \emph{is} better than the proud in spirit."],
['Ecclesiastes','Ecc','07','009','9',"Be not hasty in thy spirit to be angry: for anger resteth in the bosom of fools."],
['Ecclesiastes','Ecc','07','010','10',"Say not thou, What is \emph{the} \emph{cause} that the former days were better than these? for thou dost not enquire wisely concerning this."],
['Ecclesiastes','Ecc','07','011','11',"Wisdom \emph{is} good with an inheritance: and \emph{by} \emph{it} \emph{there} \emph{is} profit to them that see the sun."],
['Ecclesiastes','Ecc','07','012','12',"For wisdom \emph{is} a defence, \emph{and} money \emph{is} a defence: but the excellency of knowledge \emph{is,} \emph{that} wisdom giveth life to them that have it."],
['Ecclesiastes','Ecc','07','013','13',"Consider the work of God: for who can make \emph{that} straight, which he hath made crooked?"],
['Ecclesiastes','Ecc','07','014','14',"In the day of prosperity be joyful, but in the day of adversity consider: God also hath set the one over against the other, to the end that man should find nothing after him."],
['Ecclesiastes','Ecc','07','015','15',"All \emph{things} have I seen in the days of my vanity: there is a just \emph{man} that perisheth in his righteousness, and there is a wicked \emph{man} that prolongeth \emph{his} \emph{life} in his wickedness."],
['Ecclesiastes','Ecc','07','016','16',"Be not righteous over much; neither make thyself over wise: why shouldest thou destroy thyself?"],
['Ecclesiastes','Ecc','07','017','17',"Be not over much wicked, neither be thou foolish: why shouldest thou die before thy time?"],
['Ecclesiastes','Ecc','07','018','18',"\emph{It} \emph{is} good that thou shouldest take hold of this; yea, also from this withdraw not thine hand: for he that feareth God shall come forth of them all."],
['Ecclesiastes','Ecc','07','019','19',"Wisdom strengtheneth the wise more than ten mighty \emph{men} which are in the city."],
['Ecclesiastes','Ecc','07','020','20',"For \emph{there} \emph{is} not a just man upon earth, that doeth good, and sinneth not."],
['Ecclesiastes','Ecc','07','021','21',"Also take no heed unto all words that are spoken; lest thou hear thy servant curse thee:"],
['Ecclesiastes','Ecc','07','022','22',"For oftentimes also thine own heart knoweth that thou thyself likewise hast cursed others."],
['Ecclesiastes','Ecc','07','023','23',"All this have I proved by wisdom: I said, I will be wise; but it \emph{was} far from me."],
['Ecclesiastes','Ecc','07','024','24',"That which is far off, and exceeding deep, who can find it out?"],
['Ecclesiastes','Ecc','07','025','25',"I applied mine heart to know, and to search, and to seek out wisdom, and the reason \emph{of} \emph{things}, and to know the wickedness of folly, even of foolishness \emph{and} madness:"],
['Ecclesiastes','Ecc','07','026','26',"And I find more bitter than death the woman, whose heart \emph{is} snares and nets, \emph{and} her hands \emph{as} bands: whoso pleaseth God shall escape from her; but the sinner shall be taken by her."],
['Ecclesiastes','Ecc','07','027','27',"Behold, this have I found, saith the preacher, \emph{counting} one by one, to find out the account:"],
['Ecclesiastes','Ecc','07','028','28',"Which yet my soul seeketh, but I find not: one man among a thousand have I found; but a woman among all those have I not found."],
['Ecclesiastes','Ecc','07','029','29',"Lo, this only have I found, that God hath made man upright; but they have sought out many inventions."],
['Ecclesiastes','Ecc','08','001','1',"Who \emph{is} as the wise \emph{man}? and who knoweth the interpretation of a thing? a man's wisdom maketh his face to shine, and the boldness of his face shall be changed."],
['Ecclesiastes','Ecc','08','002','2',"I \emph{counsel} \emph{thee} to keep the king's commandment, and \emph{that} in regard of the oath of God."],
['Ecclesiastes','Ecc','08','003','3',"Be not hasty to go out of his sight: stand not in an evil thing; for he doeth whatsoever pleaseth him."],
['Ecclesiastes','Ecc','08','004','4',"Where the word of a king \emph{is,} \emph{there} \emph{is} power: and who may say unto him, What doest thou?"],
['Ecclesiastes','Ecc','08','005','5',"Whoso keepeth the commandment shall feel no evil thing: and a wise man's heart discerneth both time and judgment."],
['Ecclesiastes','Ecc','08','006','6',"Because to every purpose there is time and judgment, therefore the misery of man \emph{is} great upon him."],
['Ecclesiastes','Ecc','08','007','7',"For he knoweth not that which shall be: for who can tell him when it shall be?"],
['Ecclesiastes','Ecc','08','008','8',"\emph{There} \emph{is} no man that hath power over the spirit to retain the spirit; neither \emph{hath} \emph{he} power in the day of death: and \emph{there} \emph{is} no discharge in \emph{that} war; neither shall wickedness deliver those that are given to it."],
['Ecclesiastes','Ecc','08','009','9',"All this have I seen, and applied my heart unto every work that is done under the sun: \emph{there} \emph{is} a time wherein one man ruleth over another to his own hurt."],
['Ecclesiastes','Ecc','08','010','10',"And so I saw the wicked buried, who had come and gone from the place of the holy, and they were forgotten in the city where they had so done: this \emph{is} also vanity."],
['Ecclesiastes','Ecc','08','011','11',"Because sentence against an evil work is not executed speedily, therefore the heart of the sons of men is fully set in them to do evil."],
['Ecclesiastes','Ecc','08','012','12',"Though a sinner do evil an hundred times, and his \emph{days} be prolonged, yet surely I know that it shall be well with them that fear God, which fear before him:"],
['Ecclesiastes','Ecc','08','013','13',"But it shall not be well with the wicked, neither shall he prolong \emph{his} days, \emph{which} \emph{are} as a shadow; because he feareth not before God."],
['Ecclesiastes','Ecc','08','014','14',"There is a vanity which is done upon the earth; that there be just \emph{men}, unto whom it happeneth according to the work of the wicked; again, there be wicked \emph{men}, to whom it happeneth according to the work of the righteous: I said that this also \emph{is} vanity."],
['Ecclesiastes','Ecc','08','015','15',"Then I commended mirth, because a man hath no better thing under the sun, than to eat, and to drink, and to be merry: for that shall abide with him of his labour the days of his life, which God giveth him under the sun."],
['Ecclesiastes','Ecc','08','016','16',"When I applied mine heart to know wisdom, and to see the business that is done upon the earth: (for also \emph{there} \emph{is} \emph{that} neither day nor night seeth sleep with his eyes:)"],
['Ecclesiastes','Ecc','08','017','17',"Then I beheld all the work of God, that a man cannot find out the work that is done under the sun: because though a man labour to seek \emph{it} out, yet he shall not find \emph{it}; yea further; though a wise \emph{man} think to know \emph{it}, yet shall he not be able to find \emph{it}."],
['Ecclesiastes','Ecc','09','001','1',"For all this I considered in my heart even to declare all this, that the righteous, and the wise, and their works, \emph{are} in the hand of God: no man knoweth either love or hatred \emph{by} all \emph{that} \emph{is} before them."],
['Ecclesiastes','Ecc','09','002','2',"All \emph{things} \emph{come} alike to all: \emph{there} \emph{is} one event to the righteous, and to the wicked; to the good and to the clean, and to the unclean; to him that sacrificeth, and to him that sacrificeth not: as \emph{is} the good, so \emph{is} the sinner; \emph{and} he that sweareth, as \emph{he} that feareth an oath."],
['Ecclesiastes','Ecc','09','003','3',"This \emph{is} an evil among all \emph{things} that are done under the sun, that \emph{there} \emph{is} one event unto all: yea, also the heart of the sons of men is full of evil, and madness \emph{is} in their heart while they live, and after that \emph{they} \emph{go} to the dead."],
['Ecclesiastes','Ecc','09','004','4',"For to him that is joined to all the living there is hope: for a living dog is better than a dead lion."],
['Ecclesiastes','Ecc','09','005','5',"For the living know that they shall die: but the dead know not any thing, neither have they any more a reward; for the memory of them is forgotten."],
['Ecclesiastes','Ecc','09','006','6',"Also their love, and their hatred, and their envy, is now perished; neither have they any more a portion for ever in any \emph{thing} that is done under the sun."],
['Ecclesiastes','Ecc','09','007','7',"Go thy way, eat thy bread with joy, and drink thy wine with a merry heart; for God now accepteth thy works."],
['Ecclesiastes','Ecc','09','008','8',"Let thy garments be always white; and let thy head lack no ointment."],
['Ecclesiastes','Ecc','09','009','9',"Live joyfully with the wife whom thou lovest all the days of the life of thy vanity, which he hath given thee under the sun, all the days of thy vanity: for that \emph{is} thy portion in \emph{this} life, and in thy labour which thou takest under the sun."],
['Ecclesiastes','Ecc','09','010','10',"Whatsoever thy hand findeth to do, do \emph{it} with thy might; for \emph{there} \emph{is} no work, nor device, nor knowledge, nor wisdom, in the grave, whither thou goest."],
['Ecclesiastes','Ecc','09','011','11',"I returned, and saw under the sun, that the race \emph{is} not to the swift, nor the battle to the strong, neither yet bread to the wise, nor yet riches to men of understanding, nor yet favour to men of skill; but time and chance happeneth to them all."],
['Ecclesiastes','Ecc','09','012','12',"For man also knoweth not his time: as the fishes that are taken in an evil net, and as the birds that are caught in the snare; so \emph{are} the sons of men snared in an evil time, when it falleth suddenly upon them."],
['Ecclesiastes','Ecc','09','013','13',"This wisdom have I seen also under the sun, and it \emph{seemed} great unto me:"],
['Ecclesiastes','Ecc','09','014','14',"\emph{There} \emph{was} a little city, and few men within it; and there came a great king against it, and besieged it, and built great bulwarks against it:"],
['Ecclesiastes','Ecc','09','015','15',"Now there was found in it a poor wise man, and he by his wisdom delivered the city; yet no man remembered that same poor man."],
['Ecclesiastes','Ecc','09','016','16',"Then said I, Wisdom \emph{is} better than strength: nevertheless the poor man's wisdom \emph{is} despised, and his words are not heard."],
['Ecclesiastes','Ecc','09','017','17',"The words of wise \emph{men} \emph{are} heard in quiet more than the cry of him that ruleth among fools."],
['Ecclesiastes','Ecc','09','018','18',"Wisdom \emph{is} better than weapons of war: but one sinner destroyeth much good."],
['Ecclesiastes','Ecc','10','001','1',"Dead flies cause the ointment of the apothecary to send forth a stinking savour: \emph{so} \emph{doth} a little folly him that is in reputation for wisdom \emph{and} honour."],
['Ecclesiastes','Ecc','10','002','2',"A wise man's heart \emph{is} at his right hand; but a fool's heart at his left."],
['Ecclesiastes','Ecc','10','003','3',"Yea also, when he that is a fool walketh by the way, his wisdom faileth \emph{him}, and he saith to every one \emph{that} he \emph{is} a fool."],
['Ecclesiastes','Ecc','10','004','4',"If the spirit of the ruler rise up against thee, leave not thy place; for yielding pacifieth great offences."],
['Ecclesiastes','Ecc','10','005','5',"There is an evil \emph{which} I have seen under the sun, as an error \emph{which} proceedeth from the ruler:"],
['Ecclesiastes','Ecc','10','006','6',"Folly is set in great dignity, and the rich sit in low place."],
['Ecclesiastes','Ecc','10','007','7',"I have seen servants upon horses, and princes walking as servants upon the earth."],
['Ecclesiastes','Ecc','10','008','8',"He that diggeth a pit shall fall into it; and whoso breaketh an hedge, a serpent shall bite him."],
['Ecclesiastes','Ecc','10','009','9',"Whoso removeth stones shall be hurt therewith; \emph{and} he that cleaveth wood shall be endangered thereby."],
['Ecclesiastes','Ecc','10','010','10',"If the iron be blunt, and he do not whet the edge, then must he put to more strength: but wisdom \emph{is} profitable to direct."],
['Ecclesiastes','Ecc','10','011','11',"Surely the serpent will bite without enchantment; and a babbler is no better."],
['Ecclesiastes','Ecc','10','012','12',"The words of a wise man's mouth \emph{are} gracious; but the lips of a fool will swallow up himself."],
['Ecclesiastes','Ecc','10','013','13',"The beginning of the words of his mouth \emph{is} foolishness: and the end of his talk \emph{is} mischievous madness."],
['Ecclesiastes','Ecc','10','014','14',"A fool also is full of words: a man cannot tell what shall be; and what shall be after him, who can tell him?"],
['Ecclesiastes','Ecc','10','015','15',"The labour of the foolish wearieth every one of them, because he knoweth not how to go to the city."],
['Ecclesiastes','Ecc','10','016','16',"Woe to thee, O land, when thy king \emph{is} a child, and thy princes eat in the morning!"],
['Ecclesiastes','Ecc','10','017','17',"Blessed \emph{art} thou, O land, when thy king \emph{is} the son of nobles, and thy princes eat in due season, for strength, and not for drunkenness!"],
['Ecclesiastes','Ecc','10','018','18',"By much slothfulness the building decayeth; and through idleness of the hands the house droppeth through."],
['Ecclesiastes','Ecc','10','019','19',"A feast is made for laughter, and wine maketh merry: but money answereth all \emph{things}."],
['Ecclesiastes','Ecc','10','020','20',"Curse not the king, no not in thy thought; and curse not the rich in thy bedchamber: for a bird of the air shall carry the voice, and that which hath wings shall tell the matter."],
['Ecclesiastes','Ecc','11','001','1',"Cast thy bread upon the waters: for thou shalt find it after many days."],
['Ecclesiastes','Ecc','11','002','2',"Give a portion to seven, and also to eight; for thou knowest not what evil shall be upon the earth."],
['Ecclesiastes','Ecc','11','003','3',"If the clouds be full of rain, they empty \emph{themselves} upon the earth: and if the tree fall toward the south, or toward the north, in the place where the tree falleth, there it shall be."],
['Ecclesiastes','Ecc','11','004','4',"He that observeth the wind shall not sow; and he that regardeth the clouds shall not reap."],
['Ecclesiastes','Ecc','11','005','5',"As thou knowest not what \emph{is} the way of the spirit, \emph{nor} how the bones \emph{do} \emph{grow} in the womb of her that is with child: even so thou knowest not the works of God who maketh all."],
['Ecclesiastes','Ecc','11','006','6',"In the morning sow thy seed, and in the evening withhold not thine hand: for thou knowest not whether shall prosper, either this or that, or whether they both \emph{shall} \emph{be} alike good."],
['Ecclesiastes','Ecc','11','007','7',"Truly the light \emph{is} sweet, and a pleasant \emph{thing} \emph{it} \emph{is} for the eyes to behold the sun:"],
['Ecclesiastes','Ecc','11','008','8',"But if a man live many years, \emph{and} rejoice in them all; yet let him remember the days of darkness; for they shall be many. All that cometh \emph{is} vanity."],
['Ecclesiastes','Ecc','11','009','9',"Rejoice, O young man, in thy youth; and let thy heart cheer thee in the days of thy youth, and walk in the ways of thine heart, and in the sight of thine eyes: but know thou, that for all these \emph{things} God will bring thee into judgment."],
['Ecclesiastes','Ecc','11','010','10',"Therefore remove sorrow from thy heart, and put away evil from thy flesh: for childhood and youth \emph{are} vanity."],
['Ecclesiastes','Ecc','12','001','1',"Remember now thy Creator in the days of thy youth, while the evil days come not, nor the years draw nigh, when thou shalt say, I have no pleasure in them;"],
['Ecclesiastes','Ecc','12','002','2',"While the sun, or the light, or the moon, or the stars, be not darkened, nor the clouds return after the rain:"],
['Ecclesiastes','Ecc','12','003','3',"In the day when the keepers of the house shall tremble, and the strong men shall bow themselves, and the grinders cease because they are few, and those that look out of the windows be darkened,"],
['Ecclesiastes','Ecc','12','004','4',"And the doors shall be shut in the streets, when the sound of the grinding is low, and he shall rise up at the voice of the bird, and all the daughters of musick shall be brought low;"],
['Ecclesiastes','Ecc','12','005','5',"Also \emph{when} they shall be afraid of \emph{that} \emph{which} \emph{is} high, and fears \emph{shall} \emph{be} in the way, and the almond tree shall flourish, and the grasshopper shall be a burden, and desire shall fail: because man goeth to his long home, and the mourners go about the streets:"],
['Ecclesiastes','Ecc','12','006','6',"Or ever the silver cord be loosed, or the golden bowl be broken, or the pitcher be broken at the fountain, or the wheel broken at the cistern."],
['Ecclesiastes','Ecc','12','007','7',"Then shall the dust return to the earth as it was: and the spirit shall return unto God who gave it."],
['Ecclesiastes','Ecc','12','008','8',"Vanity of vanities, saith the preacher; all \emph{is} vanity."],
['Ecclesiastes','Ecc','12','009','9',"And moreover, because the preacher was wise, he still taught the people knowledge; yea, he gave good heed, and sought out, \emph{and} set in order many proverbs."],
['Ecclesiastes','Ecc','12','010','10',"The preacher sought to find out acceptable words: and \emph{that} \emph{which} \emph{was} written \emph{was} upright, \emph{even} words of truth."],
['Ecclesiastes','Ecc','12','011','11',"The words of the wise \emph{are} as goads, and as nails fastened \emph{by} the masters of assemblies, \emph{which} are given from one shepherd."],
['Ecclesiastes','Ecc','12','012','12',"And further, by these, my son, be admonished: of making many books \emph{there} \emph{is} no end; and much study \emph{is} a weariness of the flesh."],
['Ecclesiastes','Ecc','12','013','13',"Let us hear the conclusion of the whole matter: Fear God, and keep his commandments: for this \emph{is} the whole \emph{duty} of man."],
['Ecclesiastes','Ecc','12','014','14',"For God shall bring every work into judgment, with every secret thing, whether \emph{it} \emph{be} good, or whether \emph{it} \emph{be} evil."]
        
]

def main():
    ProcessVerses('Ecclesiastes03',Ecclesiastes03)

if __name__ == "__main__":
    main()



%%%%%%%%%%%%%%%%%%%%%%%%%%%
Chapter with Latex format
%%%%%%%%%%%%%%%%%%%%%%%%%%%


\footnote{\textcolor[cmyk]{0.99998,1,0,0}{\hyperlink{TOC}{Return to end of Table of Contents.}}}\textcolor[cmyk]{0.99998,1,0,0}{To every \emph{thing} \emph{there} \emph{is} a season, and a time to every purpose under the heaven:}
[2] \textcolor[cmyk]{0.99998,1,0,0}{A time to be born, and a time to die; a time to plant, and a time to pluck up \emph{that} \emph{which} \emph{is} planted;}
[3] \textcolor[cmyk]{0.99998,1,0,0}{A time to kill, and a time to heal; a time to break down, and a time to build up;}
[4] \textcolor[cmyk]{0.99998,1,0,0}{A time to weep, and a time to laugh; a time to mourn, and a time to dance;}
[5] \textcolor[cmyk]{0.99998,1,0,0}{A time to cast away stones, and a time to gather stones together; a time to embrace, and a time to refrain from embracing;}
[6] \textcolor[cmyk]{0.99998,1,0,0}{A time to get, and a time to lose; a time to keep, and a time to cast away;}
[7] \textcolor[cmyk]{

put & 1 \\ \hline
refrain & 1 \\ \hline
rejoice & 2 \\ \hline
rend & 1 \\ \hline
requireth & 1 \\ \hline
righteous & 1 \\ \hline
righteousness & 1 \\ \hline
said & 2 \\ \hline
saw & 1 \\ \hline
season & 1 \\ \hline
see & 2 \\ \hline
seen & 1 \\ \hline
set & 1 \\ \hline
sew & 1 \\ \hline
shall & 4 \\ \hline
should & 3 \\ \hline
silence & 1 \\ \hline
so & 3 \\ \hline
sons & 3 \\ \hline
speak & 1 \\ \hline
spirit & 2 \\ \hline
stones & 2 \\ \hline
sun & 1 \\ \hline
taken & 1 \\ \hline
than & 1 \\ \hline
that & 20 \\ \hline
the & 24 \\ \hline
their & 1 \\ \hline
them & 3 \\ \hline
themselves & 1 \\ \hline
there & 3 \\ \hline
they & 3 \\ \hline
thing & 2 \\ \hline
time & 31 \\ \hline
to & 37 \\ \hline
together & 1 \\ \hline
travail & 1 \\ \hline
turn & 1 \\ \hline
under & 2 \\ \hline
unto & 1 \\ \hline
up & 2 \\ \hline
upward & 1 \\ \hline
vanity & 1 \\ \hline
war & 1 \\ \hline
weep & 1 \\ \hline
what & 1 \\ \hline
whatsoever & 1 \\ \hline
wherein & 1 \\ \hline
which & 5 \\ \hline
who & 1 \